# Quantile Hedging
### Przemysław Adamski, Michał Dąbrowski, Wiktor Jacaszek

## Wprowadzenie
Celem poniższego projektu jest omówienie projektu wykonanego w ramach przedmiotu Inżynieria finansowa 2 na Uniwersytecie Wrocławskim. Polegał on na omówieniu i zaimplementowaniu do zagadnienia Quantile Hedging w zadanym świecie. Poniżej znajduje się wprowadzenie oznaczeń aktywów, które będą dalej stosowane w projekcie.

$$ \begin{align*} 
dS_t &=  \mu S_tdt + \sigma S_tdW_t, & S_0 &= 100, \\ 
dX_t &=  \widetilde{\mu} X_tdt + \widetilde{\sigma} X_td\widetilde{W_t}, & X_0 &= 100, \\ 
dB_t &=  rB_tdt, & B_0 &= 1, \\ 
\end{align*}$$
takie, że $\{(W_t, \widetilde{W_t}):t \geq 0\}$ jest dwuwymiarowym ruchem Browna ze stałą korelacją $\rho$. Wobec czego procesy stochastyczne $S_t $oraz $W_t$, są skorelowanymi geometrycznymi ruchami Browna. Wobec czego zadany świat jest opisywany przez model Blacka-Scholesa. Główną osią badań projektu będzie badanie klasycznych opcji call i put, na aktywo $S$ o cenie strike $K$ z momentem zapadania $T = 1.$ Quantile hedging polega na próbie zabezpieczenia danych opcji z kapitałem początkowym $V \leq V^{BS},$ gdzie $V^{BS}$ to cena opcji w modelu Blacka-Scholesa. W dalszej części będziemy oznaczać przez parametr $\alpha$ stosunek $\frac{V}{V^{BS}}$  
Naszym zadaniem będzie maksymalizowanie pewnych funkcji celu:
1. $\mathbb{E} \left[ \mathbb{1}_{H \leq V_T} \right] = \mathbb{P} \left [ \text{Będziemy w stanie zabezpieczyć opcję} \right] $
2. $\mathbb{E} \left[ \mathbb{1}_{H \leq V_T}  + \frac{V_T}{H} \mathbb{1}_{H > V_T} \right]$,
gdzie $H$ jest wartością wypłaty danej opcji w momencie $T$.

Warto zwrócić uwagę, że obie te funkcje są ograniczone z góry przez 1.

Główną ideą hedgingu kwantylowego, jest próba zabezpieczenia opcji z mniejszą ilością kapitału, dzięki czemu będziemy w stanie ją zabezpieczyć na pewnym poziomie istotności. Możliwe, że w takim przypadku dodaniem niewielkiej niepewności, będziemy w stanie więcej zarobić i być bardziej konkurencyjni na rynku. Dodatkowo w świecie rzeczywistym ceny aktywów nie są odzwzorywane przez geometryczny ruch Browna oraz wartości dryfu i zmienności są zmienne w czasie, zatem i tak nie jesteśmy w stanie w pełni zabezpieczyć opcji, zatem w niektórych przypadkach dodatkowe ryzyko może być pożądane, względem dodatkowego zysku uzyskanego mniejszym wkładem w hedgowanie. Na początku będziemy rozważać sytuację w której aktywo $S$ jest handlowalne, dla obu funkcji celu, następnie przejdziemy do przypadku w którym jedynie będziemy mogli handlować aktywem $X$. Będziemy oczywiście rozważać jedynie strategie samofinansujące. Naszym głównym zadaniem będzie znajdowanie maksymalnej wartości funkcji celu dla ustalonego kapitału oraz minimalnego kapitału dla ustalonej wartości funkcji celu, okaże się, że w praktyce sprowadzi się to do jednego problemu, zbadania zależności między $V \text{ i } P_V^{\max}$, gdzie $P_V^{\max}$ oznacza maksymalną funkcję celu dla kapitału początkowego $V$. Poniższy projekt został zaimplementowany w języku *Python* przy użyciu metod programowania obiektowego, podczas tworzenia korzystaliśmy z narzędzia *Git* w celu kontroli wersji, poniżej znajduje się link do  [repozytorium projektu](https://github.com/michalsn13/Quantile-Hedging).


### Zaimportowanie bibliotek/klas przed przejściem dalej <font color='red'>** (oraz lista potencjalnych instalacji!) **</font> 

In [1]:
## Importy

#Biblioteki do potencjalnego zainstalowania
#pip install matplotlib seaborn tqdm numpy pandas scipy multiprocess

from underlying import Underlying, NonTradedUnderlying
from option import Option, Vanilla, Vanilla_on_NonTraded
from trader import Trader, trader_loop
from additional_analysis_functions import dP_dQ, integrand, G_func, get_coefficients, get_delta, wrapping_function, find_x, plot_line_G
from quantile_hedging_calculator import *

import matplotlib.pyplot as plt
import seaborn as sb
from tqdm import tqdm
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import lognorm
import scipy.integrate as integrate
from scipy.optimize import root_scalar
from IPython import display
from time import sleep

from multiprocess import Pool
processors = 8

import warnings
warnings.filterwarnings("ignore")

## Wyprowadzenie hedgingu kwantylowego (dla handlowalnego aktywa bazowego)

Wyprowadźmy teraz teorię stojącą za hedgingiem kwantylowym, skupiąc się na pierwszej funkcji celu

$$\phi = \mathbb{1}_{\{H \leq V_{T} \}},$$

a więc sytuacji, w której chcielibyśmy, aby nasz ograniczony kapitał przekazany na hedging zabezpieczał nas dokładnie z jak największym prawdopodobieństwem. Aby to przedstawić formalniej oznaczmy

$$A = {\{H \leq V_{T} \}}$$

i naszym celem jest znalezienie zbioru $A$ maksymalizującego prawdopodobieństwo 

$$P \left[ A \right],$$

przy założeniu, ze początkowy kapitał $V_{0}$ spełnia $V_{0} < H_{0}$ (gdzie $H_{0}$ oznacza cenę Blacka-Scholesa zabezpieczanej opcji), oraz znalezienie strategii, którą mamy się kierować, aby cel ten osiągnąć.

#### Twierdzenie:
Jeśli $Q$ oznacza jedyną miarę martyngałową i wprowadzimy miarę $Q^{*}$, zadaną przez pochodną Radona-Nikodyma:

$$\frac{dQ^{*}}{dQ} = \frac{H}{H_{0}}$$

(gdzie H jest zmienną losową określającą payoff zabezpieczanej opcji) to zbiorem $\tilde{A}$ maksymalizującym powyższe prawdopodobieństwo jest

$$\tilde{A} = \Big{\{} \frac{dP}{dQ} > \tilde{a} \cdot H \Big{\}},$$

dla

$$\tilde{a} = inf \Big{\{} a: Q^{*} \left( \frac{dP}{dQ} > a \cdot H \right) \leq \alpha \Big{\}},$$

spełniający

$$Q^{*}(\tilde{A}) = \alpha := \frac{V_{0}}{H_{0}},$$

a optymalną strategią jest idealny delta hedging opcji o payoffie $\tilde{H} = H \cdot \mathbb{1}_{\tilde{A}}$.

Zamiast przepisywać dowód z pracy Petera Leukerta i Hansa Folmera, skupimy się na obrazowym wyjaśnieniu powyższej teorii i pokazaniu jak w praktyce znaleźć zbiór $\tilde{A}$ oraz poszukiwaną strategię.  

### Szukanie zbioru $\tilde{A}$

Zgodnie z powyższym, musimy znaleźć zbiór

$$\tilde{A} = \Big{\{} \frac{dP}{dQ} > \tilde{a} \cdot H \Big{\}}$$

Wiemy, że w przypadku modelowania zachowania aktywa geometrycznym ruchem Browna, pochodna Radona-Nikodyma ma postać

$$ \frac{d Q}{d P} =  e^{- \lambda W_{t} - \frac{1}{2} \lambda^{2}t}$$

dla $\lambda = \frac{\mu - r}{\sigma}$, tak więc występująca w naszym zbiorze odwrotność wynosi

$$ \frac{d P}{d Q} =  e^{\lambda W_{t} + \frac{1}{2} \lambda^{2}t}.$$

Skupimy się teraz na przypadku klasycznej opcji call, o cenie wykonania $K$, co daje nam payoff

$$H = \left( S_{T} - K \right)_{+}$$. Zbiór $\tilde{A}$ jest więc trajektorii spełniających nierówność

$$e^{\lambda W_{t} + \frac{1}{2} \lambda^{2}t} > \tilde{a} \cdot \left( S_{T} - K \right)_{+},$$

co po rozpisaniu wzoru $S_{T}$ daje

$$ e^{\lambda W_{T} + \frac{1}{2} \lambda^{2}T} > \tilde{a} \cdot \left( e^{ \left( \mu - \frac{\sigma^{2}}{2} \right)T + \sigma W_{T}} - K \right)_{+}.$$

Obrazowo, mamy w tej chwili (z perspektywy $W_{T}$) dwie funkcje wykładnicze i jedyny nierozpisany jeszcze parametr $\tilde{a}$, będzie służył do przemnożenia prawej strony, ale zbiór rozwiązań nierówności miał miarę $\alpha$. Po chwili zastanowienia jednak, zauważamy, że są dwa przypadki, wynikające ze współczynników przy $W_T$. Przedstawmy to na wykresie:

In [ ]:
mu = 0.02
sigma = 0.1
r = 0.01
T = 1
X0 = 100
K = 110
p = 0.8
l = (mu-r)/sigma

WT = np.arange(-2,12,0.01)
ST = X0*np.exp((mu-sigma**2/2)*T+sigma*WT)
X = np.zeros(shape=(len(ST),2))
X[:,0], X[:,1] = X0, ST
X = pd.DataFrame(X)
dP_dQ = np.exp(l*WT+0.5*l**2*T)
K = 100

fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_figwidth(10)

ax1.set_xlabel('$\mathregular{W_T}$')
ax1.set_ylabel('Payoff', labelpad=20)
ax2.set_xlabel('$\mathregular{W_T}$')
#ax2.set_ylabel('Payoff', labelpad=20)

# Payoff klasycznej funkcji call
ax1.plot(WT,np.array((X.iloc[:,-1] - K) * (X.iloc[:,-1] > K)), color = 'blue', linewidth = 3, label = 'Przeskalowany payoff' )
ax1.plot(WT, X0*dP_dQ/2, color = 'orange', linewidth = 3, label = 'Pochodna Radona-Nikodyma')
ax1.set_ylim(top=max(X0*dP_dQ)+2)
ax1.set_title('Przypadek $\mathregular{\lambda \leq \sigma}$')
ax1.set_ylim(bottom=-10,top=200)
#wybacz kreski od ręki xd
ax1.axvline(x=6.7, color='red')
ax1.legend(loc="upper left")

mu_2 = 0.032
l_2 = (mu_2-r)/sigma
WT_2 = np.arange(-2,12,0.01)
ST_2 = X0*np.exp((mu_2-sigma**2/2)*T+sigma*WT)
X_2 = np.zeros(shape=(len(ST_2),2))
X_2[:,0], X_2[:,1] = X0, ST_2
X_2 = pd.DataFrame(X_2)
dP_dQ_2 = np.exp(l_2*WT_2+0.5*l_2**2*T)

# Payoff procentowej opcji call
ax2.plot(WT_2,np.array((X_2.iloc[:,-1] - K) * (X_2.iloc[:,-1] > K)), color = 'blue', linewidth = 3, label = 'Przeskalowany payoff')
ax2.plot(WT_2, X0*dP_dQ_2/5, color = 'orange', linewidth = 3, label = 'Pochodna Radona-Nikodyma')
ax2.set_title('Przypadek $\mathregular{\lambda > \sigma}$')
ax2.set_ylim(bottom=-10,top=200)
ax2.axvline(x=3.2, color='red')
ax2.axvline(x=9.4, color='red')
ax2.legend(loc="upper left")

plt.show()

Otrzymujemy więc przypadki $\lambda \leq \sigma$ i $\lambda > \sigma$, które rozpiszemy.

W pierwszy przypadku, nierówność jest spełniona dla $W_{T}$ mniejszych niż punkt przecięcia widoczny na lewym wykresie (nazwijmy go $w_{0}$). Finalnie jednak będzie nas interesowało jaka cena aktywa wynika z takiej wartości (aby nałożyć indykator na payoff opcji), tak więc od razu możemy podejść do zagadnienia przez pryzmat ceny aktywa. Ponadto, widząc już, że otrzymamy w tym przypadku rezultat postaci $\tilde{A} = \Big{\{} W_{T} < w_{0} \Big{\}} = \Big{\{} S_{T} < c \Big{\}}$ (gdzie c to cena wynikajaca z wartości $w_{0}$) możemy oderwać się od rozwiązywania nierówności i spojrzeć dalej. Mianowicie, wiemy, że $c$ musi być dobrane tak, by cena opcji o payoffie o zerowych wartościach powyżej $c$, miała cenę $V_{0}$. Dzięki temu możemy iteracyjnie podejść do zagadnienia następująco:

1) Generujemy trajektorie $W_{T}$ i sortujemy:

$$W_{T} = \left[ -2.782, -2.23, ... ,-0.56, 0.124, ... , 2.857 \right]$$

2) Przekładamy je na ceny $S_{T}$

$$S_{T} = \left[ 58.39, 62.34, ... , 91.15, 103.14, ... , 183.73 \right]$$

3) Chcemy $Q^{*}(\tilde{A}) = \alpha$, a skoro $Q^{*}(\tilde{A}) = \mathbb{E}_{Q} \left[ \frac{dQ^{*}}{dQ} \cdot \mathbb{1}_{\tilde{A}} \right] = \mathbb{E} \left[ \frac{dQ}{dP} \cdot \frac{dQ^{*}}{dQ} \cdot \mathbb{1}_{\tilde{A}} \right]$, to likczymy najpierw pełen wektor $\frac{dQ}{dP} \cdot \frac{dQ^{*}}{dQ} (= \frac{dQ^{*}}{dP})$ zgodnie ze wzorami wyprowadzonymi wcześniej, podstawiając powyższe symulacje i payoff opcji call pod $H$:
$$\frac{dQ^{*}}{dP} = \left[ 0 , 0, ... , 0, 0.001, 0.003, ..., 4.75\right]$$

4) Iterujemy po liczbach $c$ od ostatniej ceny $S_{T}$, co skutkuje zerowaniem wartości $S_{T} > c$ oraz odpowiadających im wartości w wektorze $\frac{dQ^{*}}{dP}$, licząc przy tym $\mathbb{E} \left[ \frac{dQ}{dP} \cdot \frac{dQ^{*}}{dQ} \cdot \mathbb{1}_{\tilde{A}} \right]$, aż znajdziemy takie, które daje $\mathbb{E} \left[ \frac{dQ}{dP} \cdot \frac{dQ^{*}}{dQ} \cdot \mathbb{1}_{\tilde{A}} \right] = \alpha$. 

5) Ponieważ symulacje same z siebie wychodzą się zgodnie z rozkładem normalnym, to liczenie $\mathbb{E} \left[ \frac{dQ}{dP} \cdot \frac{dQ^{*}}{dQ} \cdot \mathbb{1}_{\tilde{A}} \right]$ odbywa się poprzez sumowanie $\frac{dQ^{*}}{dP} \cdot \mathbb{1}_{ \{ S_{T} < c \} } \cdot \frac{1}{n}$, gdzie $n$ oznacza liczbę symulacji i reprezentuje wagi (rozkłozone jednostajnie) z wartości oczekiwanej

Koniec, znaliźliśmy poszukiwane $c$ i mamy teraz "obciety" payoff naszej wyjściowej opcji call. Strategia hedgingu polega na idealnym hedgowaniu opcji, mającej włąśnie ten ocięty payoff. 

Przypadek drugi jest bardzo adekwatny. Różnica polega na tym, ze będziemy mieli dwa przedziały:
$$\tilde{A} = \Big{\{} S_{T} < c_{1} \Big{\}} \cup \Big{\{} S_{T} > c_{2} \Big{\}}$$
i do rozwiazania podchodzimy tymi samymi krokami, z tą różnicą, ze camiast iterować jednym c, to iterujemy za pomocą $c_{1}$ i $c_{2}$, by zerować wartości między nimi, zachowując jednak zasadzę, że zostawione niezerowe wartości mają takie same wagi (czyli sumy wartośći z $\frac{dQ^{*}}{dP}$, dla wartości odpowiadajacych $S_{T} < c_{1}$ i $S_{T} > c_{2}$ sa takie same.

Na koniec przedstawmy jak koncepcyjnie wyglądają payoffy opcji, które chcemy zabezpieczać:

In [ ]:
ST_1 = np.arange(80,120,1)
ST_2 = np.arange(120,150,1)
K = 100

fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_figwidth(10)

ax1.set_xlabel('$\mathregular{S_T}$')
ax1.set_ylabel('Payoff', labelpad=20)
ax2.set_xlabel('$\mathregular{S_T}$')
#ax2.set_ylabel('Payoff', labelpad=20)

# Payoff klasycznej funkcji call
ax1.plot(ST_1,(ST_1 - K) * (ST_1 > K), color = 'green', linewidth = 3, label = 'Obcięty payoff')
ax1.plot(ST_2,np.zeros(len(ST_2)) , linewidth = 3, color = 'green')
ax1.set_title('Payoff opcji do zabezpieczania, $\mathregular{\lambda \leq \sigma}$')
ax1.set_ylim(bottom=-2,top=50)
ax1.legend(loc="upper left")

ST_3 = np.arange(80,105,1)
ST_4 = np.arange(105,130,1)
ST_5 = np.arange(130,150,1)

# Payoff procentowej opcji call
ax2.plot(ST_3,(ST_3 - K) * (ST_3 > K), color = 'green', linewidth = 3, label = 'Obcięty payoff')
ax2.plot(ST_4,np.zeros(len(ST_4)) , color='green', linewidth = 3)
ax2.plot(ST_5,ST_5 - K , color='green', linewidth = 3)
ax2.set_title('Payoff opcji do zabezpieczania, $\mathregular{\lambda > \sigma}$')
ax2.set_ylim(bottom=-2,top=50)
ax2.legend(loc="upper left")

plt.show()

In [ ]:
mu = 0.06
sigma =  0.2
r = 0.00
T = 1
X0 = 100
repeat = 1000
#repeat = 10 # aby zobaczyć czy kod działa i jest sensownie
values_per_year = 250

## Aktywo handlowalne, wyniki symulacji
Poniżej przedstawimy wykresy przedstawiające wyniki otrzymane przy pomocy symulacji, dla parametru $V = 0.5 V^{BS}$ danej opcji. Na potrzebę symulacji uznaliśmy za sukces w którym po wykonaniu opcji mamy więcej niż $-0.1 V$ kapitału, wynika to z faktu, że nawet przy pełnym zabezpieczaniu przez to, że rehedgujemy nasz portfel w sposób dyskretny, a nie ciągły wartość portela może wynosić ujemną wartość, a nie 0. Tak jak wcześniej pokazaliśmy w obydwóch przypadkach funkcji celu powinniśmy się zabezpieczać w ten sam sposób, wobec czego będziemy badać je równocześnie, wartość pierwszej funkcji celu będziemy oznaczać jako *prawdopodobieństwo sukcesu*, natomiast drugiej jako *succes ratio*. Symulacje rozdzielimy na dwa przypadki w zależności od przypadku $\mu - r > \sigma^2$ oraz $\mu - r < \sigma^2$

### Przypadek $\mu - r > \sigma^2$
Poniższe wyniki otrzymaliśmy dla poniższych parametrów:
* $ \mu = 0.06$
* $ \sigma = 0.2$
* $r = 0 $

Parametry zostały tak dobrane, aby zachodził warunek $\mu - r > \sigma^2$

In [ ]:
K = 80
underlying = Underlying(mu, sigma, r, values_per_year)
_, reality = underlying.simulate_P(repeat, T)
vanilla_call = Vanilla(underlying, K, T, True)
BS_price_call = vanilla_call.get_price(X0, 0)
V0 = BS_price_call / 2
money_time_call = pd.DataFrame(np.zeros(reality.shape))
delta_time_call = pd.DataFrame(np.zeros(reality.shape))
for i in range(repeat):
    trader = Trader(initial_capital = V0)
    money, delta, objective_func = trader.simulate_hedging(vanilla_call, X0*reality.iloc[[i],:], update_freq = 1, mode = 'quantile_traded')
    money_time_call.loc[i] = money
    delta_time_call.loc[i] = delta
#print(f'Success probability with initial capital={V0:.2f} is: {objective_func[0]:.4f}')
#print(f'Success ratio with initial capital={V0:.2f} is: {objective_func[1]:.4f}')


### Opcja call
Poniżej znajdują się wykresy dla opcji z ceną wykonanania $K = 80$, $V = 10.59$ 

In [ ]:
X = pd.DataFrame(np.arange(0,300))
plt.figure(figsize=(10,5))
vanilla_call = Vanilla(underlying, K, T, True)
BS_price = vanilla_call.get_price(X0, 0)
V0 = 0.5 * BS_price
payoff_func, objective_func, c = payoff_from_v0(vanilla_call, V0, X0)
plt.plot(X, payoff_func(X))
plt.title(f'K = {K}\np-stwo sukcesu = {objective_func[0]:.3f} i success ratio = {objective_func[1]:.3f}')
plt.axvline(K, color = 'black', lw = 4);

In [ ]:
money_time_call_outcome = money_time_call.copy()
money_time_call_outcome['outcome'] = money_time_call.apply(lambda row: 'success' if row.iloc[-1] >= - 0.1 * V0 else 'fail', axis = 1)
fig, axs = plt.subplots(4,1, figsize = (10, 20))
sb.histplot(ax = axs[0],x = money_time_call_outcome[underlying.values_per_year * T], bins = 50, hue = money_time_call_outcome['outcome'], legend = True, stat = 'density').set_title(f'Ostateczny stan portfela- wszystkie symulacje');
sb.histplot(ax = axs[1], x = (X0*reality)[underlying.values_per_year * T], hue = money_time_call_outcome['outcome'], bins = 50, stat = 'density').set_title('Ostateczna cena aktywa bazowego- wszystkie symulacje')
axs[1].axvline(K, color = 'black', lw = 4)
sb.scatterplot(ax = axs[2], x = (X0*reality).iloc[:,-1], y = np.maximum((X0*reality)[underlying.values_per_year * T] - K,0), hue = money_time_call_outcome['outcome']).set_title('Ostateczny payoff opcji (wszystkie symulacje)')
sb.histplot(ax = axs[3], x = np.maximum((X0*reality)[underlying.values_per_year * T] - K,0), hue = money_time_call_outcome['outcome'], bins = 50, stat = 'density').set_title('Ostateczny payoff opcji (wszystkie symulacje)- histogram');

### Opcja put
Poniżej znajdują się wykresy dla opcji z ceną wykonanania $K = 120$, $V = 11.07$

In [ ]:

K = 120
underlying = Underlying(mu, sigma, r, values_per_year)
_, reality = underlying.simulate_P(repeat, T)
vanilla_put = Vanilla(underlying, K, T, False)
BS_price_put = vanilla_put.get_price(X0, 0)
money_time_put = pd.DataFrame(np.zeros(reality.shape))
delta_time_put = pd.DataFrame(np.zeros(reality.shape))
V0 = BS_price_put / 2
for i in range(repeat):
    trader = Trader(initial_capital = V0)
    money, delta, objective_func = trader.simulate_hedging(vanilla_put, X0*reality.iloc[[i],:], update_freq = 1, mode = 'quantile_traded')
    money_time_put.loc[i] = money
    delta_time_put.loc[i] = delta

In [ ]:
X = pd.DataFrame(np.arange(0,300))
plt.figure(figsize=(10,5))
vanilla_put = Vanilla(underlying, K, T, False)
BS_price = vanilla_put.get_price(X0, 0)
V0 = 0.5 * BS_price
payoff_func, objective_func, c = payoff_from_v0(vanilla_put, V0, X0)
plt.plot(X, payoff_func(X))
plt.title(f'K = {K}\np-stwo sukcesu = {objective_func[0]:.3f} i success ratio = {objective_func[1]:.3f}')
plt.axvline(K, color = 'black', lw = 4);

In [ ]:
money_time_put_outcome = money_time_put.copy()
money_time_put_outcome['outcome'] = money_time_put.apply(lambda row: 'success' if row.iloc[-1] >= - 0.1 * V0 else 'fail', axis = 1)
fig, axs = plt.subplots(4,1, figsize = (10, 20))
sb.histplot(ax = axs[0],x = money_time_put_outcome[underlying.values_per_year * T], bins = 50, hue = money_time_put_outcome['outcome'], legend = True, stat = 'density').set_title(f'Ostateczny stan portfela- wszystkie symulacje');
sb.histplot(ax = axs[1], x = (X0*reality)[underlying.values_per_year * T], hue = money_time_put_outcome['outcome'], bins = 50, stat = 'density').set_title('Ostateczna cena aktywa bazowego- wszystkie symulacje')
axs[1].axvline(K, color = 'black', lw = 4)
sb.scatterplot(ax = axs[2], x = (X0*reality).iloc[:,-1], y = np.maximum( K -(X0*reality)[underlying.values_per_year * T],0), hue = money_time_put_outcome['outcome']).set_title('Ostateczny payoff opcji (wszystkie symulacje)')
sb.histplot(ax = axs[3], x = np.maximum(K - (X0*reality)[underlying.values_per_year * T],0), hue = money_time_put_outcome['outcome'], bins = 50, stat = 'density').set_title('Ostateczny payoff opcji (wszystkie symulacje)- histogram');

### Przypadek $\mu - r < \sigma^2$
Poniższe wyniki otrzymaliśmy dla poniższych parametrów:
* $ \mu = 0.06$
* $ \sigma = 0.2$
* $r = 0.05 $

Parametry zostały tak dobrane, aby zachodził warunek $\mu - r < \sigma^2$

In [ ]:
r = 0.05

### Opcja call
Poniżej znajdują się wykresy dla opcji z ceną wykonanania $K = 80$, $V = 12.29$

In [ ]:
K = 80
underlying = Underlying(mu, sigma, r, values_per_year)
_, reality = underlying.simulate_P(repeat, T)
vanilla_call = Vanilla(underlying, K, T, True)
BS_price_call = vanilla_call.get_price(X0, 0)
V0 = BS_price_call / 2
money_time_call = pd.DataFrame(np.zeros(reality.shape))
delta_time_call = pd.DataFrame(np.zeros(reality.shape))
for i in range(repeat):
    trader = Trader(initial_capital = V0)
    money, delta, objective_func = trader.simulate_hedging(vanilla_call, X0*reality.iloc[[i],:], update_freq = 1, mode = 'quantile_traded')
    money_time_call.loc[i] = money
    delta_time_call.loc[i] = delta
#print(f'Success probability with initial capital={V0:.2f} is: {objective_func[0]:.4f}')
#print(f'Success ratio with initial capital={V0:.2f} is: {objective_func[1]:.4f}')

In [ ]:
X = pd.DataFrame(np.arange(0,300))
plt.figure(figsize=(10,5))
vanilla_call = Vanilla(underlying, K, T, True)
BS_price = vanilla_call.get_price(X0, 0)
V0 = 0.5 * BS_price
payoff_func, objective_func, c = payoff_from_v0(vanilla_call, V0, X0)
plt.plot(X, payoff_func(X))
plt.title(f'K = {K}\np-stwo sukcesu = {objective_func[0]:.3f} i success ratio = {objective_func[1]:.3f}')
plt.axvline(K, color = 'black', lw = 4);

In [ ]:
money_time_call_outcome = money_time_call.copy()
money_time_call_outcome['outcome'] = money_time_call.apply(lambda row: 'success' if row.iloc[-1] >= - 0.1 * V0 else 'fail', axis = 1)
fig, axs = plt.subplots(4,1, figsize = (10, 20))
sb.histplot(ax = axs[0],x = money_time_call_outcome[underlying.values_per_year * T], bins = 50, hue = money_time_call_outcome['outcome'], legend = True, stat = 'density').set_title(f'Ostateczny stan portfela- wszystkie symulacje');
sb.histplot(ax = axs[1], x = (X0*reality)[underlying.values_per_year * T], hue = money_time_call_outcome['outcome'], bins = 50, stat = 'density').set_title('Ostateczna cena aktywa bazowego- wszystkie symulacje')
axs[1].axvline(K, color = 'black', lw = 4)
sb.scatterplot(ax = axs[2], x = (X0*reality).iloc[:,-1], y = np.maximum((X0*reality)[underlying.values_per_year * T] - K,0), hue = money_time_call_outcome['outcome']).set_title('Ostateczny payoff opcji (wszystkie symulacje)')
sb.histplot(ax = axs[3], x = np.maximum((X0*reality)[underlying.values_per_year * T] - K,0), hue = money_time_call_outcome['outcome'], bins = 50, stat = 'density').set_title('Ostateczny payoff opcji (wszystkie symulacje)- histogram');

### Opcja put
Poniżej znajdują się wykresy dla opcji z ceną wykonanania $K = 120$, $V = 8.69$

In [ ]:
K = 120
underlying = Underlying(mu, sigma, r, values_per_year)
_, reality = underlying.simulate_P(repeat, T)
vanilla_put = Vanilla(underlying, K, T, False)
BS_price_put = vanilla_put.get_price(X0, 0)
money_time_put = pd.DataFrame(np.zeros(reality.shape))
delta_time_put = pd.DataFrame(np.zeros(reality.shape))
V0 = BS_price_put / 2
for i in range(repeat):
    trader = Trader(initial_capital = V0)
    money, delta, objective_func = trader.simulate_hedging(vanilla_put, X0*reality.iloc[[i],:], update_freq = 1, mode = 'quantile_traded')
    money_time_put.loc[i] = money
    delta_time_put.loc[i] = delta

In [ ]:
X = pd.DataFrame(np.arange(0,300))
plt.figure(figsize=(10,5))
vanilla_put = Vanilla(underlying, K, T, False)
BS_price = vanilla_put.get_price(X0, 0)
V0 = 0.5 * BS_price
payoff_func, objective_func, c = payoff_from_v0(vanilla_put, V0, X0)
plt.plot(X, payoff_func(X))
plt.title(f'K = {K}\np-stwo sukcesu = {objective_func[0]:.3f} i success ratio = {objective_func[1]:.3f}')
plt.axvline(K, color = 'black', lw = 4);

In [ ]:
money_time_put_outcome = money_time_put.copy()
money_time_put_outcome['outcome'] = money_time_put.apply(lambda row: 'success' if row.iloc[-1] >= - 0.1 * V0 else 'fail', axis = 1)
fig, axs = plt.subplots(4,1, figsize = (10, 20))
sb.histplot(ax = axs[0],x = money_time_put_outcome[underlying.values_per_year * T], bins = 50, hue = money_time_put_outcome['outcome'], legend = True, stat = 'density').set_title(f'Ostateczny stan portfela- wszystkie symulacje');
sb.histplot(ax = axs[1], x = (X0*reality)[underlying.values_per_year * T], hue = money_time_put_outcome['outcome'], bins = 50, stat = 'density').set_title('Ostateczna cena aktywa bazowego- wszystkie symulacje')
axs[1].axvline(K, color = 'black', lw = 4)
sb.scatterplot(ax = axs[2], x = (X0*reality).iloc[:,-1], y = np.maximum( K -(X0*reality)[underlying.values_per_year * T],0), hue = money_time_put_outcome['outcome']).set_title('Ostateczny payoff opcji (wszystkie symulacje)')
sb.histplot(ax = axs[3], x = np.maximum(K - (X0*reality)[underlying.values_per_year * T],0), hue = money_time_put_outcome['outcome'], bins = 50, stat = 'density').set_title('Ostateczny payoff opcji (wszystkie symulacje)- histogram');

Na podstawie powyższych ilustracji jesteśmy w stanie zauważyć, że używając quantile hedgingu faktycznie zabezpieczamy się na opcje, z payoffem znajdującym się na korespondującym wykresie. Warto zwrócić uwagę, że histogramy przedstawiające sukcesy i porażki nachodzą na siebie, co wynika z wad delta hedgingu. Istnieją nawet symulacje których payoff wynosi 0, jednak nie byliśmy w stanie się na nie zabezpieczyć przez sposób zachowywania się portfela.

## Analiza wrażliwości

Poniżej przeprowadzę podstawową analizę wpływu parametrów $K, \sigma \text{ i } \alpha$ na kształt funkcji zmodyfikowanego payoffu oraz wartości funkcji celu. Domyślnymi argumentami wartości funkcji jest cena strike $K = 100, \sigma = 0.2, \alpha =0.5 \text{ oraz } r = 0.05$. Przeprowadzimy rozumowanie dla obu opcji, call i put.

## wpływ ceny strike $K$

In [ ]:
Ks = [ 80, 100, 120, 140, 160, 180]
X = pd.DataFrame(np.arange(0,300))
fig, axs = plt.subplots(3, 2, figsize=(10,10), sharex = True, sharey = True)
fig.tight_layout(pad=5.0)
for num, K in enumerate(Ks):
    vanilla_call = Vanilla(underlying, K, T, True)
    BS_price = vanilla_call.get_price(X0, 0)
    V0 = 0.5 * BS_price
    payoff_func, objective_func, c = payoff_from_v0(vanilla_call, V0, X0)
    axs[num//2, num%2].plot(X, payoff_func(X))
    axs[num//2, num%2].set_title(f'K = {K}\np-stwo sukcesu = {objective_func[0]:.3f} i success ratio = {objective_func[1]:.3f}')
    axs[num//2, num%2].axvline(K, color = 'black', lw = 4)
plt.subplots_adjust(top=0.9)
plt.suptitle('Kwantylowy hedging opcji call używając połowy ceny w zależności od ceny wykonania K\nzmodyfikowany payoff', size = 12);

In [ ]:
Ks = [80, 100, 120, 140, 160, 180]
X = pd.DataFrame(np.arange(0,300))
fig, axs = plt.subplots(3, 2, figsize=(10,10), sharex = True, sharey = True)
fig.tight_layout(pad=5.0)
for num, K in enumerate(Ks):
    vanilla_put = Vanilla(underlying, K, T, False)
    BS_price = vanilla_put.get_price(X0, 0)
    V0 = 0.5 * BS_price
    payoff_func, objective_func, c = payoff_from_v0(vanilla_put, V0, X0)
    axs[num//2, num%2].plot(X, payoff_func(X))
    axs[num//2, num%2].set_title(f'K = {K}\np-stwo sukcesu = {objective_func[0]:.3f} i success ratio = {objective_func[1]:.3f}')
    axs[num//2, num%2].axvline(K, color = 'black', lw = 4)
plt.subplots_adjust(top=0.9)
plt.suptitle('Kwantylowy hedging opcji put używając połowy ceny w zależności od ceny wykonania K\nzmodyfikowany payoff', size = 12);

Z powyższych rysunków możemy zaobserwować, że zmiana jedynie parametru $K$ zmienia prawdopobieństwo zabezpieczenia opcji, w przypadku opcji call wzrost parametru $K$ zwiększa je, natomiast w przypadku opcji put zmniejsza. Wynika to między innymi z faktu, że wzrost parametru $K$ zwiększa prawdopodobieństwo, że wartość payoffu opcji call wyniesie 0, natomiast w przypadku opcji put zwiększa prawdopodobieństwo na dodatni payoff.

## Wpływ parametru $\alpha$

In [ ]:
K = 100
alphas = [0.01, 0.1, 0.2, 0.4, 0.6, 0.8, 0.9, 0.99]
X = pd.DataFrame(np.arange(0,300))
fig, axs = plt.subplots(4, 2, figsize=(10,10), sharex = True, sharey = True)
fig.tight_layout(pad=5.0)
for num, alpha in enumerate(alphas):
    vanilla_call = Vanilla(underlying, K, T, True)
    BS_price = vanilla_call.get_price(X0, 0)
    V0 = alpha * BS_price
    payoff_func, objective_func, _ = payoff_from_v0(vanilla_call, V0, X0)
    axs[num//2, num%2].plot(X, payoff_func(X))
    axs[num//2, num%2].set_title(f'alpha = {alpha}\np-stwo sukcesu = {objective_func[0]:.3f} i success ratio = {objective_func[1]:.3f}')
    axs[num//2, num%2].axvline(K, color = 'black', lw = 4)
plt.subplots_adjust(top=0.9)
plt.suptitle(f'Kwantylowy hedging opcji call(K = {K}) w zależności od procenta użytej ceny\nzmodyfikowany payoff', size = 12);

In [ ]:
K = 100
alphas = [0.01, 0.1, 0.2, 0.4, 0.6, 0.8, 0.9, 0.99]
X = pd.DataFrame(np.arange(0,300))
fig, axs = plt.subplots(4, 2, figsize=(10,10), sharex = True, sharey = True)
fig.tight_layout(pad=5.0)
vanilla_put = Vanilla(underlying, K, T, False)
BS_price = vanilla_put.get_price(X0, 0)
for num, alpha in enumerate(alphas):
    V0 = alpha * BS_price
    payoff_func, objective_func, c = payoff_from_v0(vanilla_put, V0, X0)
    axs[num//2, num%2].plot(X, payoff_func(X))
    axs[num//2, num%2].set_title(f'alpha = {alpha}\np-stwo sukcesu = {objective_func[0]:.3f} i success ratio = {objective_func[1]:.3f}')
    axs[num//2, num%2].axvline(K, color = 'black', lw = 4)
plt.subplots_adjust(top=0.9)
plt.suptitle(f'Kwantylowy hedging opcji put(K = {K}) w zależności od procenta użytej ceny\nzmodyfikowany payoff', size = 12);

Powyższe wykresy wskazują na to, że wzraz ze wzrostem parametru $\alpha$, zabezpieczamy się na opcję o większym payoffie, jest to dość oczywisty i intuicyjny wniosek, ponieważ nasz kapitał się zwiększa, zatem możemy zabezpieczyć się na droższą opcję. Warto zwróciv uwagę, że nawet dla wartości $\alpha$ bliskich 0, prawdopodobieństwo zabezpieczenia opcji jest spore.

## Wpływ parametru $\sigma $


In [ ]:
K = 100
sigmas = [0.05, 0.1, 0.2, 0.5, 0.6, 0.7, 0.8, 0.9]
X = pd.DataFrame(np.arange(0,300))
fig, axs = plt.subplots(4, 2, figsize=(10,10), sharex = True, sharey = True)
fig.tight_layout(pad=5.0)
for num, s in enumerate(sigmas):
    underlying = Underlying(mu, s, r, values_per_year)    
    vanilla_call = Vanilla(underlying, K, T, True)
    BS_price = vanilla_call.get_price(X0, 0)
    V0 = 0.5 * BS_price
    payoff_func, objective_func, _ = payoff_from_v0(vanilla_call, V0, X0)
    axs[num//2, num%2].plot(X, payoff_func(X))
    axs[num//2, num%2].set_title(f'sigma = {s}\np-stwo sukcesu = {objective_func[0]:.3f} i success ratio = {objective_func[1]:.3f}')
    axs[num//2, num%2].axvline(K, color = 'black', lw = 4)
plt.subplots_adjust(top=0.9)
plt.suptitle(f'Kwantylowy hedging opcji call(K = {K}) używając połowy ceny w zależności od zmienności aktywa\nzmodyfikowany payoff', size = 12);

In [ ]:
K = 100
sigmas = [0.05, 0.1, 0.2, 0.5, 0.6, 0.7, 0.8, 0.9]
X = pd.DataFrame(np.arange(0,300))
fig, axs = plt.subplots(4, 2, figsize=(10,10), sharex = True, sharey = True)
fig.tight_layout(pad=5.0)
for num, s in enumerate(sigmas):
    underlying = Underlying(mu, s, r, values_per_year)    
    vanilla_put = Vanilla(underlying, K, T, False)
    BS_price = vanilla_put.get_price(X0, 0)
    V0 = 0.5 * BS_price
    payoff_func, objective_func, _ = payoff_from_v0(vanilla_put, V0, X0)
    axs[num//2, num%2].plot(X, payoff_func(X))
    axs[num//2, num%2].set_title(f'sigma = {s}\np-stwo sukcesu = {objective_func[0]:.3f} i success ratio = {objective_func[1]:.3f}')
    axs[num//2, num%2].axvline(K, color = 'black', lw = 4)
plt.subplots_adjust(top=0.9)
plt.suptitle(f'Kwantylowy hedging opcji put(K = {K}) używając połowy ceny w zależności od zmienności aktywa\nzmodyfikowany payoff', size = 12);

Możemy zauważyć, z rysunków że $\sigma$ ma wpływ na kształt zmodyfikowanego payoffu, co jest zgodne z warunkiem na kształt w zależności od wartości znaku wyrażenia $\mu - r - \sigma^2 $. Wraz ze wzrostem zmienności, prawdopodobieństwo zabezpieczenia opcji call wzrasta używając połowy kapitału, natomiast dla opcji put zależność jest odwrotna.

## Wpływ parametru $\alpha \text{ oraz } K$

In [ ]:
Ks = [80, 100, 120, 140]
fig, axs = plt.subplots(2, 2, figsize=(10,10))
for num, K in enumerate(Ks):
    vanilla_call = Vanilla(underlying, K, T, True)
    BS_price = vanilla_call.get_price(X0, 0)
    alphas = np.arange(5,101,1)/100
    probs = []
    ratios = []
    for a in alphas:
        _, objective_func, _ = payoff_from_v0(vanilla_call, a * BS_price, X0)
        probs.append(objective_func[0])
        ratios.append(objective_func[1])
    axs[num//2, num%2].plot(alphas, probs, color = 'black')
    axs[num//2, num%2].plot(alphas, ratios, color = 'blue')
    axs[num//2, num%2].legend(['p-stwo sukcesu', 'success ratio'])
    axs[num//2, num%2].plot([0,1],[0,1], color = 'red', lw = 1, linestyle='-.')
    axs[num//2, num%2].set_title(f'Cena pełna opcji = {BS_price:.2f} z K = {K}')
    axs[num//2, num%2].set_xlabel('alpha')
    axs[num//2, num%2].set_ylabel('funkcja celu')
    axs[num//2, num%2].set_xlim([0.05,1])
    axs[num//2, num%2].set_ylim([0.05,1])
plt.suptitle('Hedging kwantylowy opcji call w zależności od ceny wykonania K\nalpha vs wartość funkcji celu', size = 15);

In [ ]:
Ks = [80, 100, 120, 140]
fig, axs = plt.subplots(2, 2, figsize=(10,10))
for num, K in enumerate(Ks):
    vanilla_put = Vanilla(underlying, K, T, False)
    BS_price = vanilla_put.get_price(X0, 0)
    alphas = np.arange(5,101,1)/100
    probs = []
    ratios = []
    for a in alphas:
        _, objective_func, _ = payoff_from_v0(vanilla_put, a * BS_price, X0)
        probs.append(objective_func[0])
        ratios.append(objective_func[1])
    axs[num//2, num%2].plot(alphas, probs, color = 'black')
    axs[num//2, num%2].plot(alphas, ratios, color = 'blue')
    axs[num//2, num%2].legend(['p-stwo sukcesu', 'success ratio'])
    axs[num//2, num%2].plot([0,1],[0,1], color = 'red', lw = 1, linestyle='-.')
    axs[num//2, num%2].set_title(f'Cena pełna opcji = {BS_price:.2f} z K = {K}')
    axs[num//2, num%2].set_xlabel('alpha')
    axs[num//2, num%2].set_ylabel('funkcja celu')
    axs[num//2, num%2].set_xlim([0.05,1])
    axs[num//2, num%2].set_ylim([0.05,1])
plt.suptitle('Hedging kwantylowy opcji putw zależności od ceny wykonania K\nalpha vs wartość funkcji celu', size = 15);

Jedną z zależności którą możemy zaobserwować z wykresu, jest ta, że wze wzrostem parametru $\alpha$ zwiększa się wartość funkcji celu, wynika to z tego że zabezpieczamy się większą  opcją, wobec czego jesteśmy zabezpieczeni na więcej scenariuszy. Dodatkowo znowu możemy dojść do wniosku, że wzrost ceny strike zwiększa prawdopodobieństwo zabezpieczenia dla opcji call, natomiast dla opcji put zmniejsza. Warto zwrócić uwagę na dość duże prawdopodbieństwo zabezpieczenia dla opcji call, i znacznie mniejsze dla opcji put.

### Druga Funkcja Celu

Rozpatrzymy teraz drugą funkcję celu

$$ \phi = \mathbb{1}_{\{H \leq V_{T} \}} + \frac{V_{T}}{H} \mathbb{1}_{\{H > V_{T} \}}$$

i naszym celem ponownie będzie optymalizacja jej wartości oczekiwanej $ \left( \mathbb{E}\left[ \phi \right] \right) $.
Publikacja Hansa F \"{o} llmera i Petera Leukerta tym razem także dostarcza rozwiązanie, według którego strategią optymalizującą powyższą wartość jest pełny $\Delta$-hedging opcji o payoffie $\tilde{H} = \tilde{\phi}H$, dla

$$\tilde{\phi} = \mathbb{1}_{\{\frac{dP}{dQ} > \tilde{a} H \}} + \gamma \mathbb{1}_{\{\frac{dP}{dQ} = \tilde{a} H \}},$$
gdzie

$$\gamma = \frac{\alpha - Q^{*} \left[ \frac{dP}{dQ} > \tilde{a}H  \right]}{Q^{*} \left[ \frac{dP}{dQ} = \tilde{a}H  \right]}.$$

Pierwszy indykator funkcji $\tilde{\phi}$ pokrywa sytuacje z pierwszej funkcji celu, tak więc jedyną różnicą jest występowanie drugiego indykatora.

Teza:

W przypadku klasycznych opcji call i put oraz trajektorii modelowanych geometrycznym ruchem Browna, zbiór $\{\frac{dP}{dQ} = \tilde{a} H \}$ jest zbiorem miary zero.

Dowód:

Jak już wiemy

$$ \frac{d P}{d Q} =  e^{\lambda W_{t} + \frac{1}{2} \lambda^{2}t}.$$ 

Tym samym rozpatrujemy zbiór trajektorii spełniających równość

$$e^{\lambda W_{T} + \frac{1}{2} \lambda^{2}T} = \tilde{a} \cdot H.$$

Korzystamy w tym miejscu z założenia, że rozpatrujemy opcję call (w przypadku opcji put argument jest analogiczny) i możemy rozpisać payoff $H$ jako:

$$H = \left( S_{T} - K \right)_{+}.$$

Następnie rozpisujemy jawnie wzór na $S_{T}$, co daje

$$H = \left( e^{ \left( \mu - \frac{\sigma^{2}}{2} \right)T + \sigma W_{T}} - K \right)_{+} $$

czym dochodzimy do poniższej postaci równości 

$$ e^{\lambda W_{T} + \frac{1}{2} \lambda^{2}T} = \tilde{a} \cdot \left( e^{ \left( \mu - \frac{\sigma^{2}}{2} \right)T + \sigma W_{T}} - K \right)_{+}.$$

Zauważmy teraz, że możemy pominąć nałożoną po prawej stronie część dodatnią. Lewa strona równości jest funkcją wykładniczą, tak więc zawsze jest dodatnia i tym samym równość nie może zajść w przypadku gdy prawa strona jest zerowa lub ujemna, wobec czego pozwolenie ujemnym przypadkom pozostać ujemnym (zamiast zamieniania ich na 0), nie zmienia zbioru rozwiązań. Mamy tym samym

$$ e^{\lambda W_{T} + \frac{1}{2} \lambda^{2}T} = \tilde{a} \cdot  \left( e^{ \left( \mu - \frac{\sigma^{2}}{2} \right)T + \sigma W_{T}} - K \right).$$

Rozpiszmy teraz $\lambda$

$$ e^{ \left( \frac{\mu - r}{\sigma} \right) W_{T} + \frac{1}{2} \left( \frac{\mu - r}{\sigma} \right)^{2}T} = \tilde{a} \cdot e^{ \left( \mu - \frac{\sigma^{2}}{2} \right)T + \sigma W_{T}} - \tilde{a} \cdot K$$

oraz dla przejrzystości wprowadźmy dwa oznaczenia

$$b_{1} = e^{\frac{1}{2} \left( \frac{\mu - r}{\sigma} \right)^{2}T}\ , \quad b_{2} = e^{\left( \mu - \frac{\sigma^{2}}{2} \right)T}$$

by ostatecznie otrzymać

$$ b_{1} \cdot e^{ \left( \frac{\mu - r}{\sigma} \right) W_{T}} = \tilde{a} \cdot b_{2} \cdot e^{\sigma W_{T}} - \tilde{a} \cdot K.$$

Są teraz trzy możliwe sytuacje:
1. Równanie jest sprzeczne i nie ma rozwiązań, co zadaje zbiór pusty (który jest oczywiście miary zero)

2. Równanie ma jedno lub dwa rozwiązania. Korzystamy teraz jednak z założenia o ciągłości, a mianowicie, ponieważ $W_{T}$ pochodzi z rozkładu normalnego, to dla każdego $x$ mamy $P \left [W_{T} = x \right] = 0 $, co również daje nam zbiór miary zero.

3. Potencjalnie interesowałaby nas więc jedynie ostatnia sytuacja  w której funkcje się (przynajmniej na pewnym przedziale) pokrywają, co zadałoby przedział o niezerowej mierze. Tak się jednak nie dzieje, ponieważ o ile może się zdarzyć, że części wykładnicze są takie same, o tyle prawa strona ma niezerowe przesunięcie. Uzasadnienie:

    Przesuniecie jest zadane przez $\tilde{a} K$. Jeśli $\tilde{a} = 0$, to cała prawa strona równa się $0$, co w obliczu dodatniości lewej strony daje sprzeczność. $K$ natomiast jest parametrem opcji call. Z założenia K > 0, ponieważ w przeciwnym przypadku nie mielibyśmy do czynienia z tym aktywem pochodnym (w przypadku K = 0 opcja dawałaby nam możliwość "zakupu" aktywa za darmo, niezależnie od scenariusza).
    
Co kończy dowód, że $\{\frac{dP}{dQ} = \tilde{a} H \}$ jest miary 0.

Wniosek:

Optymalizacja drugiej funkcji celu jest osiągana przez strategię, którą realizowaliśmy przy pierwszej funkcji celu.

## Druga funkcja celu z niezerową miarą drugiego indykatora

Naszym celem jest teraz wykreować sytuację, w której drugi indykator nie będzie bazował na zbiorze o zerowej mierze. Dążymy więc do tego, by zbiór

$$D = \Big{\{} \frac{dP}{dQ} = \tilde{a} H \Big{\}}$$

miał dodatnią miarę. Kluczowym momentem w poprzednim dowodzie było doprowadzenie powyższego zbioru do postaci

$$D = \Big{\{} e^{\lambda W_{T} + \frac{1}{2} \lambda^{2}T} = \tilde{a} \cdot H \Big{\}},$$

a zerowość wynikała z payoffu klasycznej funkcji call, a konkretnie odejmowania współczynnika, którego nie można było wyzerować. Oznacza to, że aby osiagnąć cel musimy znaleźć opcję o innym payoffie, a powyższy dowodów sugeruje, że zapewne udałoby się dopasować w niej współczynniki jeśli jej payoff będzie miał charakter multiplikatywny. Wprowadzamy więc taką opcję, którą nazwyamy autorsko Procentową Opcją Call. Przedstawiamy jej działanie poniżej.

**Procentowa Opcja Call:** <br>
Posiadacz opcji zyskuje możliwość zakupu w chwili $T$ aktywa po cenie $p \cdot S_{T}$, jeśli cena przekroczy próg $K$ $(\text{dla }p \in (0,1))$. <br>

**Payoff:** $H = \left( S_{T} - p \cdot S_{T} \right) \cdot \mathbb{1}_{\{ S_{T} > K \}} = \left( 1 - p \right) \cdot S_{T}  \cdot \mathbb{1}_{\{ S_{T} > K \}}$

Spójrzmy jak on wygląda (na każdym kroku porównując do klasycznej opcji call):

In [ ]:
mu = 0.02
sigma = 0.1
r = 0.01
T = 1
X0 = 100
K = 110
p = 0.8
l = (mu-r)/sigma
values_per_year = 250

def measure_payoff(X,p,K):
    return (1-p)*X.iloc[:, -1]*(X.iloc[:, -1] > K)

#Przedstawienie jak wyglada payoff tej funkcji z perspektywy S_T i porównanie do klasycznej
ST = np.arange(70,141,1)
X = np.zeros(shape=(len(ST),2))
X[:,0], X[:,1] = X0, ST
X = pd.DataFrame(X)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_figwidth(10)

ax1.set_xlabel('$\mathregular{S_T}$')
ax1.set_ylabel('Payoff', labelpad=20)
ax2.set_xlabel('$\mathregular{S_T}$')

# Payoff klasycznej funkcji call
ax1.plot(ST,np.array((X.iloc[:,-1] - K) * (X.iloc[:,-1] > K)), color = 'purple', linewidth = 3, label='H')
ax1.set_ylim(top=max(max((X.iloc[:,-1] - K) * (X.iloc[:,-1] > K)),max(measure_payoff(X.iloc[ST>K,:],p,K)))+2)
ax1.set_ylim(bottom = -2)
ax1.set_title('Klasyczna opcja call, K = ' + str(K))
ax1.legend(loc="upper left")
ax1.set_xticks(np.arange(70,141,10))

# Payoff procentowej opcji call
ax2.plot(ST[ST<=K],np.array(measure_payoff(X.iloc[ST<=K,:],p,K)), color = 'blue', linewidth = 3, label='H')
ax2.plot(ST[ST>K],np.array(measure_payoff(X.iloc[ST>K,:],p,K)), color = 'blue', linewidth = 3)
ax2.set_ylim(top=max(max((X.iloc[:,-1] - K) * (X.iloc[:,-1] > K)),max(measure_payoff(X.iloc[ST>K,:],p,K)))+2)
ax2.set_ylim(bottom = -2)
ax2.set_title('Procentowa opcja call, K = ' + str(K) + ', p = ' +  str(p))
ax2.legend(loc="upper left")
ax2.set_xticks(np.arange(70,141,10))

plt.show()


Zobaczmy, że taka opcja rzeczywiście może spełniać powyższy warunek. Podstawiamy za $H$ policzony payoff i otrzymujemy

$$D = \Big{\{} e^{\lambda W_{T} + \frac{1}{2} \lambda^{2}T} = \tilde{a} \cdot \left( 1 - p \right) \cdot S_{T} \cdot \mathbb{1}_{\{ S_{T} > K \}}  \Big{\}}$$

oraz po rozpisaniu $S_{T}$

$$D = \Big{\{} e^{\lambda W_{T} + \frac{1}{2} \lambda^{2}T} = \tilde{a} \cdot \left( 1 - p \right) \cdot e^{\left(\mu - \frac{\sigma^{2}}{2}\right)T + \sigma W_{T}} \cdot \mathbb{1}_{\{ S_{T} > K \}}  \Big{\}}.$$

Intuicja za powyższym warunkiem $\left( D = \Big{\{} \frac{dP}{dQ} = \tilde{a} H \Big{\}} \right)$ jest następująca: chcemy przeskalować payoffu opcji, w sposób, który umożliwia jej pokrycie się z pochodną Radona-Nikodyma na pewnym przedziale (w tym przypadku $S_{T} < K$). Sytuację przed i po przeskalowanu możemy zaobserwowac na poniższych wykresach: (Uwaga! w literaturze patrzy się na ceny z perspektywy stosunku do wartości początkowej aktywa, tu jednak pozostaniemy przy wartościach nominalnych, tak więc pochodna Radona-Nikodyma została przemnożona przez cenę początkową wynoszącą 100)

In [ ]:
WT = np.arange(-2,8,0.01)
ST = X0*np.exp((mu-sigma**2/2)*T+sigma*WT)
X = np.zeros(shape=(len(ST),2))
X[:,0], X[:,1] = X0, ST
X = pd.DataFrame(X)

#plt.plot(WT,np.array(a_wave*measure_payoff(X,p,1.1)))
#Radon-Nikodym
dP_dQ = np.exp(l*WT+0.5*l**2*T)

In [ ]:
################
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_figwidth(10)

ax1.set_xlabel('$\mathregular{W_T}$')
ax1.set_ylabel('Payoff', labelpad=20)
ax2.set_xlabel('$\mathregular{W_T}$')
#ax2.set_ylabel('Payoff', labelpad=20)

# Payoff klasycznej funkcji call
ax1.plot(WT,np.array((X.iloc[:,-1] - K) * (X.iloc[:,-1] > K)), color = 'blue', linewidth = 3)
ax1.plot(WT, X0*dP_dQ, color = 'orange', linewidth = 3)
ax1.set_ylim(top=max(X0*dP_dQ)+2)
ax1.set_title('Klasyczna opcja call')

# Payoff procentowej opcji call
ax2.plot(WT[ST<=K],np.array(measure_payoff(X.iloc[ST<=K,:],p,K)), color = 'blue', linewidth = 3)
ax2.plot(WT[ST>K],np.array(measure_payoff(X.iloc[ST>K,:],p,K)), color = 'blue', linewidth = 3)
ax2.plot(WT, X0*dP_dQ, color = 'orange', linewidth = 3)
ax2.set_title('Procentowa opcja call')
ax2.set_ylim(top=max(X0*dP_dQ)+2)
plt.show()

In [ ]:
#Przeskalowane przez a_wave
a_wave = np.exp((sigma**2-mu)*T)/(1-p)
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_figwidth(10)

ax1.set_xlabel('$\mathregular{W_T}$')
ax1.set_ylabel('Payoff', labelpad=20)
ax2.set_xlabel('$\mathregular{W_T}$')
#ax2.set_ylabel('Payoff', labelpad=20)

# Payoff klasycznej funkcji call
ax1.plot(WT,a_wave*np.array((X.iloc[:,-1] - K) * (X.iloc[:,-1] > K)), color = 'blue', linewidth = 3)
ax1.plot(WT, X0*dP_dQ, color = 'orange', linewidth = 3)
ax1.set_ylim(top=max(X0*dP_dQ)+2)
ax1.set_title('Klasyczna opcja call')
ax1.plot()

# Payoff procentowej opcji call
ax2.plot(WT[ST<=K],np.array(a_wave*measure_payoff(X.iloc[ST<=K,:],p,K)), color = 'blue', linewidth = 3)
ax2.plot(WT[ST>K],np.array(a_wave*measure_payoff(X.iloc[ST>K,:],p,K)), color = 'green', linewidth = 3)
ax2.plot(WT[ST<=K], X0*dP_dQ[ST<=K], color = 'orange', linewidth = 3)
ax2.plot(WT[ST>K], X0*dP_dQ[ST>K], color = 'green', linewidth = 3)
ax2.set_title('Procentowa opcja call')
ax2.set_ylim(top=max(X0*dP_dQ)+2)
plt.show()


Przejdźmy więc do konkretnego doboru parametrów. Ponieważ $W_{T}$ jest traktowany jako *zmienna* tych funkcji, to w celu zapewnienia równości (na przedziale $S_{T} > K$) musimy zapewnić równość współczynników przy $W_{T}$ oraz tych mnożących funkcję wykładniczą. równość współczynników $W_{T}$ daje kluczowy warunek:

$$\lambda = \sigma,$$

co po rozpisaniu $\lambda = \frac{\mu-r}{\sigma}$ daje

$$\mu - r = \sigma^{2}.$$

Możemy teraz skrócić cześci potęg związanych z $W_{T}$ oraz użyć powyższego warunku do podstawienia pod $\lambda$ i przenieść eksponens na jedną stronę, aby orzymać

$$D = \Big{\{} e^{\left(\sigma^{2} - \mu \right)T} = \tilde{a} \cdot \left( 1 - p \right) \cdot \mathbb{1}_{\{ S_{T} > K \}}  \Big{\}}.$$

Problematyczną kwestią zostaje wartość parametru $\tilde{a}$, ponieważ nie wiemy, czy nasz warunek spowodował, że powyżej rzeczywiście może zachodzi równość. Przyjrzyjmy się więc temu. Z definicji mamy

$$\tilde{a} = inf \Big{\{} a: Q^{*} \left( \frac{dP}{dQ} > a \cdot H \right) \leq \alpha \Big{\}},$$

podstawiamy $\frac{dP}{dQ}$ i $H$

$$\tilde{a} = inf \Big{\{} a: Q^{*} \left( e^{\lambda W_{T} + \frac{1}{2} \lambda^{2}T} > a \cdot \left( 1 - p \right) \cdot S_{T} \cdot \mathbb{1}_{\{ S_{T} > K \}} \right) \leq \alpha \Big{\}},$$

ponownie rozpisujemy $S_{T}$

$$\tilde{a} = inf \Big{\{} a: Q^{*} \left( e^{\lambda W_{T} + \frac{1}{2} \lambda^{2}T} > a \cdot \left( 1 - p \right) \cdot e^{\left(\mu - \frac{\sigma^{2}}{2}\right)T + \sigma W_{T}} \cdot \mathbb{1}_{\{ S_{T} > K \}} \right) \leq \alpha \Big{\}},$$

i korzystamy z warunku $\lambda = \sigma$ oraz grupujemy

$$\tilde{a} = inf \Big{\{} a: Q^{*} \left( e^{\left(\sigma^{2} - \mu \right)T} > a \cdot \left( 1 - p \right) \cdot \mathbb{1}_{\{ S_{T} > K \}} \right) \leq \alpha \Big{\}}$$

i ostatnie przekształcenie

$$\tilde{a} = inf \Big{\{} a: Q^{*} \left( \frac{e^{\left(\sigma^{2} - \mu \right)T}}{a \cdot \left( 1 - p \right)} > \mathbb{1}_{\{ S_{T} > K \}} \right) \leq \alpha \Big{\}}.$$

Wprowadźmy oznaczenie

$$l(a) := \frac{e^{\left(\sigma^{2} - \mu \right)T}}{a \cdot \left( 1 - p \right)}$$

Szukamy więc ograniczenia dolnego dla a, spośród tych, dla których miara rozwiązań nierówności

$$l(a) > \mathbb{1}_{\{ S_{T} > K \}}$$

w mierze $Q^{*}$ nie przekroczy $\alpha$ (które jest ostro mniejsze od 1).

Obrazowo:

In [ ]:
ST = np.arange(70,170,1)
K = 110

fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_figwidth(10)

ax1.set_xlabel('$\mathregular{S_T}$')
ax1.set_ylabel('Payoff', labelpad=20)
ax2.set_xlabel('$\mathregular{S_T}$')
#ax2.set_ylabel('Payoff', labelpad=20)

# Infimum w klasycznej opcji call
ax1.plot(ST[ST<=K],np.zeros(len(ST[ST<=K])), color = 'blue', linewidth = 3, label = '1-K/$\mathregular{S_T}$')
ax1.plot(ST[ST>=K],1-K/ST[ST>=K], color = 'blue', linewidth = 3)
ax1.plot(ST,0.2*np.ones(len(ST)), color = 'orange', linewidth = 3, label='l(a)')
ax1.set_title('Infimum w klasycznej opcji call')
ax1.set_xlabel('$\mathregular{S_T}$')
ax1.set_ylim([-0.1,1.1])
ax1.legend(loc="center left")

# Infimum w klasycznej opcji call
ax2.plot(ST[ST<=K],np.zeros(len(ST[ST<=K])), color = 'blue', linewidth = 3, label='Indykator $\mathregular{1_{S_T > K}}$' )
ax2.plot(ST[ST>=K],np.ones(len(ST[ST>=K])), color = 'blue', linewidth = 3)
ax2.plot(ST,0.2*np.ones(len(ST)), color = 'orange', linewidth = 3, label='l(a)')
ax2.set_title('Infimum w procentowej opcji call')
ax2.set_xlabel('$\mathregular{S_T}$')
ax2.set_ylim([-0.1,1.1])
ax2.legend(loc="center left")
plt.show()

#plt.plot(ST[ST<=K],np.zeros(len(ST[ST<=K])), color = 'blue', linewidth = 3, label='Indykator')
#plt.plot(ST[ST>=K],np.ones(len(ST[ST>=K])), color = 'blue', linewidth = 3)
#plt.plot(ST,0.7*np.ones(len(ST)), color = 'orange', linewidth = 3, label='l(a)')
#plt.xlabel('$\mathregular{S_T}$')
#plt.ylabel('Wartość', labelpad=20)
#plt.title('Poszukiwanie infimum')
#plt.legend(loc="upper left")

Widzimy więc, ze mamy trzy przypadki:

1. Jeśli $l(a) < 0$ to $Q^{*} \left( l(a) > \mathbb{1}_{\{ S_{T} > K \}} \right) = 0$ 

2. Jeśli $l(a) \geq 1$ to $Q^{*} \left( l(a) > \mathbb{1}_{\{ S_{T} > K \}} \right) = 1$

3. Jeśli $l(a) \in [0,1)$ to $Q^{*} \left( l(a) > \mathbb{1}_{\{ S_{T} > K \}} \right) = Q^{*} \left(S_{T} \leq K \right)$

Okazuje się jednak, że zachodzi: 
$$Q^{*} \left(S_{T} > K \right) = 0$$

Dowód:
$$Q^{*} \left(S_{T} \leq K \right) = \mathbb{E}_{Q^{*}} \left[ \mathbb{1}_{\{ S_{T} \leq K \}} \right] = \mathbb{E}_{Q} \left[ \frac{dQ^{*}}{dQ} \mathbb{1}_{\{ S_{T} \leq K \}} \right] = \mathbb{E}_{Q} \left[ \frac{H}{H_{0}} \mathbb{1}_{\{ S_{T} \leq K \}} \right] = \mathbb{E}_{Q} \left[ \frac{\left( 1 - p \right) \cdot S_{T}  \cdot \mathbb{1}_{\{ S_{T} > K \}}}{H_{0}} \mathbb{1}_{\{ S_{T} \leq K \}} \right] = 0.$$

Redukuje to nam sytuację do dwóch przypadków

1. Jeśli $l(a) < 1$ to $Q^{*} \left( l(a) > \mathbb{1}_{\{ S_{T} > K \}} \right) = 0$ 

2. Jeśli $l(a) \geq 1$ to $Q^{*} \left( l(a) > \mathbb{1}_{\{ S_{T} > K \}} \right) = 1$

A ponieważ $l(a)$ jest malejące ze względu na $a$, to infimum jest osiagane rzeczywiście gdy $l(a) = 1$, co daje upragnioną równość.

# Wyniki drugiej funkcji celu na Procentowej Opcji Call

Przypomnijmy, że druga funckja celu jest maksymalizowana przez pełny hedging opcji o payoffie $\tilde{\phi} \cdot H$ dla

$$\tilde{\phi} = \mathbb{1}_{\{ \frac{dP}{dQ} \leq \tilde{a}H \}} + \gamma \cdot \mathbb{1}_{\{ \frac{dP}{dQ} = \tilde{a}H \}},$$
gdzie
$\gamma = \frac{\alpha - Q^{*}\left( \frac{dP}{dQ} \leq \tilde{a}H \right)}{Q^{*}\left( \frac{dP}{dQ} = \tilde{a}H \right)}.$ Otrzymany rezultat upraszcza tę sytuację do

$$\gamma = \frac{\alpha - 0}{1} = \alpha$$
i w konsekwencji
$$\tilde{\phi} = \alpha \cdot \mathbb{1}_{\{ \frac{dP}{dQ} = \tilde{a}H \}} = \frac{V_{0}}{H_{0}} \cdot \mathbb{1}_{\{ \frac{dP}{dQ} = \tilde{a}H \}}$$

Oznacza to, że w przypadku tej opcji, idealnie będziemy "zabezpieczeni" jedynie na sytuacje, w których nie dojdzie do wypłaty. W przypadkach gdy do wypłaty dojdzie, będziemy natomiast zawsze tracili i naszym celem będzie minimalizowanie tej straty. Wynika na to, że strategią realizującą ten cel, będzie dokładny $\Delta$-hedging opcji, której payoff jest przemnożony przez stosunek kapitału, który chcemy przeznaczyć na hedging kwantylowy, do ceny opcji. Obrazuje to poniższy wykres:

In [ ]:
ST = np.arange(70,141,0.1)
X = np.zeros(shape=(len(ST),2))
X[:,0], X[:,1] = X0, ST
X = pd.DataFrame(X)

fig = plt.figure()
fig.set_figwidth(10)
plt.plot(ST[ST<=K],np.array(measure_payoff(X.iloc[ST<=K,:],p,K)), color = 'blue', linewidth = 3, label='Payoff opcji')
plt.plot(ST[ST>K],np.array(measure_payoff(X.iloc[ST>K,:],p,K)), color = 'blue', linewidth = 3)
plt.plot(ST[ST<=K],np.array(measure_payoff(X.iloc[ST<=K,:],p,K)), color = 'green', linewidth = 3)
plt.plot(ST[ST>K],0.8*np.array(measure_payoff(X.iloc[ST>K,:],p,K)), color = 'orange', linewidth = 3, label='Payoff do zabezpieczenia')
plt.xlabel('$\mathregular{S_T}$')
plt.ylabel('Wartość', labelpad=20)
plt.title('Payoff funkcji do hedgingu kwantylowego')
plt.legend(loc="upper left")

Mają już wyprowadzoną całą teorię, możemy spojrzeć jak rpzekłąda się ona na wyniki z symulacji. Zobaczmy więc, jak zwiększanie przekazywanego na hedging  kwantylowy kapitału, przekłada się, na funkcje celu w przypadku procentowej opcji call.

Uwaga 1: Do poniższych symulacji oraz oszacowania ceny opcji korzystamy z symulacji Monte Carlo (jest to przy okazji test, czy nasza klasa działa na "dowoloną" opcję), jednak można by użyć jawnych wzorów, ponieważ w przypadku powyższej opcji wszystkie wartości są nam znane, gdyż ich wyprowadzenie jest niemalże identyczne jak w przypadku klasycznej opcji call. Tak więc dla zainteresowanych:

Cena Procentowej Opcji Call:

$$H_{t} = (1-p) \cdot S_{0} \cdot \Phi(d_{2} + \sigma \cdot \sqrt{T-t},0,1),$$

gdzie $\Phi(x,0,1)$ oznacza dystrybuantę rozkładu normalnego N(0,1), natomiast

$$d_{2} = \frac{log{\left( \frac{S_{0}}{K} \right) + \left( r - \frac{\sigma^{2}}{2} \right) \cdot \left( T-t \right)}}{\sigma \cdot \sqrt{T-t} },$$

Wzór na Deltę:
$$\Delta_{t} = (1-p) \cdot \Phi(d_{2},0,1).$$

Uwaga 2: przyjęliśmy bardzo rygorystyczny warunek, że wyniki poniżej zera są porażkami (tak więc delta hedging dajacy przykładowo -0.05 jest wartością ujemną, a więc porażką, a nie "blisko zera = sukcesem"

Spójrzmy na wyniki symulacji:

In [ ]:
alphas = np.arange(0.15,1,0.2)

mu = 0.06
sigma =  0.2
r = 0.03
T = 1
X0 = 100
repeat = 20
values_per_year = 250
K = 110

def measure_payoff(X,p,K):
    return (1-p)*X.iloc[:, -1]*(X.iloc[:, -1] > K)

underlying = Underlying(mu, sigma, r, values_per_year)
_, reality = underlying.simulate_P(repeat, T)
option = Option(underlying,lambda X : measure_payoff(X,p,K),T)
Measure_Price = option.get_MC_price(X0)

Collection_Funkcja_Uno_Mean = []
Collection_Funkcja_Dos_Mean = []

for a in alphas:
    funkcja_uno = np.zeros(reality.shape[0])
    funkcja_dos = np.zeros(reality.shape[0])
    for i in tqdm(range(repeat)):
        trader = Trader(initial_capital = a*Measure_Price)
        dont_mind, don_care, obj = trader.simulate_hedging(option, X0*reality.iloc[[i],:], update_freq = 1)
        funkcja_uno[i] = obj[0]
        funkcja_dos[i] = obj[1]
    Collection_Funkcja_Uno_Mean.append(np.mean(funkcja_uno))
    Collection_Funkcja_Dos_Mean.append(np.mean(funkcja_dos))

In [ ]:
fig = plt.figure()
fig.set_figwidth(8,5)
fig.set_figheight(5)
plt.plot(alphas, Collection_Funkcja_Uno_Mean, color = 'black')
plt.plot(alphas, Collection_Funkcja_Dos_Mean, color = 'blue')
plt.legend(['Pierwsza funkcja celu', 'Druga funkcja celu'], loc = 'lower right')
plt.title('Funkcje celów Procentowej opcji call')
plt.xlabel('Część ceny BS')
plt.ylabel('Wartość')
plt.xlim([min(alphas),max(alphas)])
plt.ylim([0,1])
plt.show()

Możemy ogłosić sukces! Jak widzimy, dla opcji zaprojektowanej w ten sposób, zwiększanie nakładów na hedging kwantylowy rzeczywiście optymalizuje drugą funkcję celu, mimo że nie zmienia nic w wartośći pierwszej. Udało się nam więc jawnie uwypuklić, że teoria stojąca za optymalizowaniem drugiej funkcji celu działa samoistnie i nie jest implikowana przez optymalizację pierwszej funkcji celu

## Wprowadzenie do zabezpieczania opcji w przypadku gdy $S_t$ jest niehandlowalne

In [ ]:
mu = 0.06
mu_tilde = 0.06
sigma =  0.3
sigma_tilde =  0.3
r = 0.05
S0 = 100
X0_t = 100
T = 1
K = 100
repeat = 1000
values_per_year = 250
rho = 0.7

### Krótkie wprowadzenie
W przypadku opcji niehandlowalnych w celu osiągnięcia wniosków będziemy korzystać z metod monte carlo. Metoda z której korzystamy została przedstawiona na wykładzie, korzysta ona z parametru $m$, który jest przez uwikłanie związany z wartością kapitału początkowego $V$. Dla ustalonego parametru $m$ oraz wygenerowanej zmiennej $\widetilde{W_T}$ będziemy badać funkcje postaci $$ F_Q(x) =\mathbb{E}^Q[\frac{dP}{dQ} \phi_D^x| \widetilde{W_T}],$$
gdzie $\phi_D^x$ jest wartością funkcji celu dla funkcji payoffu $D$,a $x$ kapitałem portfela w momencie wykonania opcji. Z kolei $\frac{dP}{dQ}$ jest pochodną Radona - Nikodyma daną poniższym wzorem: 

$$\frac{\partial P}{\partial Q} = \exp{\dfrac{\widetilde{\mu}-r}{\widetilde{\sigma}} \widetilde{W_T} + \frac{1}{2} \left( \dfrac{ \widetilde{\mu}-r}{ \widetilde{\sigma}} \right) ^2 T}$$
Dla takiej funkcji $F_Q(x)$ będziemy dopasowywać styczną do niej, który ogranicza ją z góry. Najmniejszy punkt styczności będzie wartością portfela w momencie wykonania opcji. Przy znajdowaniu stycznej będziemy korzystać z pochodnych tych funkcji. Poniżej przedstawimy w uproszczonej formie funkcje $F_Q(x)$ oraz ich pochodne, dla obu funkcji celu, dla pierwszej funkcji celu, będziemy oznaczać ją jako $F_Q(x)$, a dla drugiej jako $G_Q(x)$. Warto zwrócić uwagę, że zarówno $F_Q(x)$, jak i $G_Q(x)$ funkcje te są ograniczone z dołu przez 0, a z góry przez $\frac{dP}{dQ} oraz są niemalejące$l.$

#### Jak znaleźć styczną?
Aby znaleźć taką styczną należy znaleźć takie argumenty $x$, aby wartości odpowiednio $F_Q'(x)$, bądź $G_Q'$ wynosiły $m$. Dla takich punktów należy sprawdzić czy styczna przechodząca przez nie ogranicza z góry, $F_Q'(x)$, bądź $G_Q'$, jeśli tak to wybierzemy taki najmniejszy $x$, który to spełnia, natomiast jeśli nie ma takiego $x$, to takiej stycznej nie ma i wartość portfela tuż przed wykonaniem opcji będzie wynosić 0.

### Pierwsza funkcja celu
#### Opcja call
$$ F_Q(x) = \frac{\partial P}{\partial Q} \Phi \left[ \frac{\log{ \frac{k+x}{S_0}}- \mu T + \frac{\sigma^2 T}{2} - \rho \sigma \widetilde{W_T}}{\sigma \sqrt{T(1-\rho^2)}}\right]$$

$$ F_Q'(x) = \frac{\partial P}{\partial Q} \frac{1}{\sqrt{2\pi}} \exp{\left( - \frac{ \left[\log{ \frac{k+x}{S_0}}- \mu T + \frac{\sigma^2 T}{2} - \rho \sigma \widetilde{W_T}\right]^2} {2 \sigma^2 T(1-\rho^2)} \right)} \frac{1}{\left(k+x\right) \left( \sigma \sqrt{T(1-\rho^2)}\right)}$$
#### Opcja put
$$ F_Q(x) = \frac{\partial P}{\partial Q} \left ( 1- \Phi \left[ \frac{\log{ \frac{k-x}{S_0}}- \mu T + \frac{\sigma^2 T}{2} - \rho \sigma \widetilde{W_T}}{\sigma \sqrt{T(1-\rho^2)}}\right] \right)$$

$$ F_Q'(x) = \frac{\partial P}{\partial Q} \frac{1}{\sqrt{2\pi}} \exp{\left( - \frac{ \left[\log{ \frac{k-x}{S_0}}- \mu T + \frac{\sigma^2 T}{2} - \rho \sigma \widetilde{W_T}\right]^2} {2 \sigma^2 T(1-\rho^2)} \right)} \frac{1}{\left(k-x\right) \left( \sigma \sqrt{T(1-\rho^2)}\right)}$$

#### Znajdywanie stycznej
W tym przypadku stosując podstawienie $u = \log{(k+x)}$ dla opcji call, $u = \log{(k-x)}$ dla opcji put, sprowadzimy zagadnienie $ F_Q'(x) = m$, do znalezienia miejsc zerowych funkcji kwadratowej. Jeśli takich miejsc nie ma to odpowiednia styczna nie istnieje, natomiast jeśli są to należy z nich $x$ stosując podstawienie odwrotne. W przypadku gdy jest jedno miejsce zerowe to należy sprawdzić czy odpowiadająca mu styczna ogranicza z góry $F_Q(x)$, dzięki właściwościom $F_Q(x)$, wystarczy to sprawdzić dla $x$, w pobliżu 0, należy również sprawdzić czy $x \geq 0$, jeśli tak to znaleźlismy wartość portfela w chwili $T$. W przypadku gdy są dwa miejsca zerowe postępujemy analogicznie, również znajdujemy odpowiednie $x$, poprzez podstawienie odwrotne, jednak z własności funkcji $F_Q(x)$, wiemy styczna przechodząca przez mniejszego $x$ nie ogranicza z góry $F_Q(x)$, zatem istnieje jedynie jeden kandydat na styczną, wobec czego należy przeprowadzić analogiczne rozumowanie do przypadku gdy istniało tylko jedno miejsce zerowe. Poniżej znajdują się wykresy przedstawiające przykładową funkcje $F_Q(x), F_Q'(x),$ dobraną styczną oraz różnice między styczną, a $F_Q(x).$

In [ ]:
x_range = 200
delta = 0.1
W_T_tilde = 3
rho = 0.7
x_call = np.arange(delta, x_range + delta, delta)
df_call = pd.DataFrame({ "x" : x_call, "G(x)" : None, "G2(x)" : None, "G2'(x)" : None})
m = 0.005
for i in range(len(x_call)):
    df_call.at[i, "G(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 1, derivative = False)
    df_call.at[i, "G2(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 2, derivative = False)
    df_call.at[i, "G'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 1, derivative = True)
    df_call.at[i, "G2'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 2, derivative = True)

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (20, 10))
axs[0].plot(df_call['x'],df_call['G(x)'], label = "$F_Q(x)$")
axs[0].set_title(" $F_Q(x)$, Opcja call, $\widetilde{\mathregular{W_T}}$ = 3, K = 100, rho = 0.7")
axs[0].set_xlabel('x')
axs[0].legend(loc='upper left')
axs[1].plot(df_call["x"],df_call["G'(x)"], label = "$F'_Q(x)$")
axs[1].set_title("$F'_Q(x)$, Opcja call, $\widetilde{\mathregular{W_T}}$ = 3, m = 0.005")
axs[1].set_xlabel('x')
axs[1].axhline(y = 0.005, color = 'r', linestyle = 'dashdot', label = "m")
axs[1].legend(loc='upper left')
plt.show()

In [ ]:
x = find_x(m, W_T_tilde, mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0,  rho, type = 'call', target_function = 1,  left_bound = 0.01,eps = 0.1 ** 10, right_bound = 100, arg_inf = 120)
df = plot_line_G(x, m, mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = 'call', x_range = 200)
fig, axs = plt.subplots(1,2, figsize = (20, 10))
axs[0].plot(df['x'], df["G(x)"], label = "$F_Q(x)$")
axs[0].plot(df['x'], df["Line"], label = "Line")
axs[0].set_xlabel('x')
axs[0].legend(loc='upper left')
axs[0].set_title(" $F_Q(x)$, Opcja call, $\widetilde{\mathregular{W_T}}$ = 3, K = 100, rho = 0.7")
axs[1].plot(df['x'], df["Diff"], label = "Line - $F_Q(x)$")
axs[1].legend(loc='upper left')
axs[0].set_xlabel('x')
plt.plot()

### Druga funkcja celu
#### Opcja call

$$a(x) = \left[ \frac{\log{ \frac{k+x}{S_0}}- \mu T + \frac{\sigma^2 T}{2} - \rho \sigma \widetilde{W_T}}{\sigma \sqrt{1-\rho^2}}\right] $$
$$ G_Q(x) = G_1(x) + \frac{\partial P}{\partial Q}  \int_{a(x)}^\infty \dfrac{x}{S_0\exp(\mu T+\sigma \widetilde{W_T} \rho+\sqrt{1-\rho^2} y \sigma - \sigma^2 T/2)-K} f_{W_T}(y)dy  $$
$$ G_Q'(x) = \frac{\partial P}{\partial Q}  \left[ \int_{a(x)}^\infty \dfrac{1}{S_0\exp(\mu T+\sigma \widetilde{W_T} \rho+\sqrt{1-\rho^2} y \sigma - \sigma^2 T/2)-K} f_{W_T}(y)dy  \right] $$

#### Opcja put

$$b(x) = \left[ \frac{\log{ \frac{k-x}{S_0}}- \mu T + \frac{\sigma^2 T}{2} - \rho \sigma \widetilde{W_T}}{\sigma \sqrt{1-\rho^2}}\right] $$
$$ G_2(x) = G_1(x) + \frac{\partial P}{\partial Q}  \int_{-\infty}^b \dfrac{x}{ K - S_0\exp(\mu T+\sigma \widetilde{W_T} \rho+\sqrt{1-\rho^2} y \sigma - \sigma^2 T/2)} f_{W_T}(y)dy $$
$$ G_2'(x) = \frac{\partial P}{\partial Q}  \int_{-\infty}^b \dfrac{1}{ K - S_0\exp(\mu T+\sigma \widetilde{W_T} \rho+\sqrt{1-\rho^2} y \sigma - \sigma^2 T/2)} f_{W_T}(y)dy $$

#### Znajdywanie stycznej
Dla drugiej funkcji celu podejście jest numeryczne zamiast analitycznego. Wynika to z faktu, że pochodna nie jest zadana w sposób jawny, tylko postaci całkowej i będziemy więc liczyć jej wartość w sposób numeryczny do tego celu posłużą nam narzędzia z biblioteki *scipy*.Z drugiej strony funkcja funkcja $G_Q'(x)$ jest funkcją malejącą wobec czego ułatwi to nam znajdywanie odpowiedniej stycznej, gwarantuje to nam istnienie jedynej stycznej. Nasz algorytm polegał na wzięciu przedziału $[a,b]$, gdzie punkt $a$ leży w pobliżu 0, a $b$ taki aby wyrażenie $G_Q'(b)-m$ było ujemne. Na początku sprawdzamy czy $G_Q'(a)$ jest większe od $m$, jeśli nie wartość portfela w chwili T wynosi 0. Natomiast jeśli tak to znajdujemy miejsce zerowe wyrażenia $G_Q'(x)-m$ przy pomocy bisekcji i jest to punkt przez który nasza styczna. Z wklęsłości $G_Q(x)$, wiemy że jej styczna ogranicza ją z góry. 

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (20, 10))
axs[0].plot(df_call['x'],df_call['G2(x)'], label = "$G_Q(x)$")
axs[0].set_title(" $G_Q(x)$, Opcja call, $\widetilde{\mathregular{W_T}}$ = 3, K = 100, rho = 0.7")
axs[0].set_xlabel('x')
axs[0].legend(loc='upper left')
axs[1].plot(df_call["x"],df_call["G2'(x)"], label = "$G'_Q(x)$")
axs[1].set_title("$G'_Q(x)$, Opcja call, $\widetilde{\mathregular{W_T}}$ = 3, K = 100, rho = 0.7 m = 0.005")
axs[1].set_xlabel('x')
axs[1].axhline(y = 0.005, color = 'r', linestyle = 'dashdot', label = "m")
axs[1].legend(loc='upper left')
plt.show()

In [ ]:
x = find_x(m, W_T_tilde, mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0,  rho, type = 'call', target_function = 2,  left_bound = 0.01,eps = 0.1 ** 10, right_bound = 100, arg_inf = 120)
df = plot_line_G(x, m, mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = 'call', x_range = 200, target_function = 2)
fig, axs = plt.subplots(1,2, figsize = (20, 10))
axs[0].plot(df['x'], df["G2(x)"], label = "$G_Q(x)$")
axs[0].plot(df['x'], df["Line"], label = "Line")
axs[0].set_xlabel('x')
axs[0].legend(loc='upper left')
axs[0].set_title(" $G_Q(x)$, Opcja call, $\widetilde{\mathregular{W_T}}$ = 3, K = 100, rho = 0.7, m =0.005")
axs[1].plot(df['x'], df["Diff"], label = "Line - $G_Q(x)$")
axs[1].set_xlabel('x')
axs[1].legend(loc='upper left')
plt.plot();

### Przykładowe funkcje $F_Q(x)$ i $G_Q(x)$

Na poniższych wykresach funkcja $F_Q(x) $ jest oznaczona kolorem niebieskim, a $G_Q(x)$ pomarańczowym.

In [ ]:
x_range_call = 200
x_range_put = K
delta = 0.1
W_T_tilde = 3
vec_W_T_tilde = [-3, -3, -1, -1, 1, 1, 3, 3]
rho = 0.7
x_call = np.arange(delta, x_range_call + delta, delta)
df_call = pd.DataFrame({ "x" : x_call, "G(x)" : None, "G2(x)" : None, "G2'(x)" : None})
x_put = np.arange(delta, K, delta)

def dP_dQ(mu, sigma, T, r, W_T): 
    return np.exp(W_T * (mu-r) / sigma + 0.5 * T * ((mu - r) / sigma) ** 2)

#### Wpływ parametru $\widetilde{W_T}$ na funkcje F(x) i G(x)

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(15,15))
Rad_Nik = dP_dQ(mu_tilde, sigma_tilde, T, r, W_T_tilde)
for num, W_T_tilde in enumerate(vec_W_T_tilde):
    if(num %2 ==0):
        df_call = pd.DataFrame({ "x" : x_call, "G(x)" : None, "G2(x)" : None, "G2'(x)" : None})
        for i in range(len(x_call)):
            df_call.at[i, "G(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 1, derivative = False)
            df_call.at[i, "G2(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 2, derivative = False)
            #df_call.at[i, "G'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 1, derivative = True)
            #df_call.at[i, "G2'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 2, derivative = True)
            axs[num//2, num%2].plot(df_call['x'], df_call["G(x)"], color = "blue", label = "F(x)", linewidth = 0.5, alpha =0.7)
            axs[num//2, num%2].plot(df_call['x'], df_call["G2(x)"], color = "orange", label = "G(x)", linewidth = 0.5, alpha =0.7)
            axs[num//2, num%2].set_ylim([0, max(1.05,dP_dQ(mu_tilde, sigma_tilde, T, r, W_T_tilde) + 0.1)])
            axs[num//2, num%2].set_xlim([0,x_range_call])
            axs[num//2, num%2].set_title('F(x) i G(x) w dla opcji call, $\widetilde{W_T}$ =' + f'{W_T_tilde}', fontsize = 10)
            
            
    else:
        df_put = pd.DataFrame({ "x" : x_put, "G(x)" : None, "G2(x)" : None, "G2'(x)" : None})
        for i in range(len(x_put)):
            df_put.at[i, "G(x)"] = G_func(df_put.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "put", target_function = 1, derivative = False)
            df_put.at[i, "G2(x)"] = G_func(df_put.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "put", target_function = 2, derivative = False)
            #df_call.at[i, "G'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 1, derivative = True)
            #df_call.at[i, "G2'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 2, derivative = True)
            axs[num//2, num%2].plot(df_put['x'], df_put["G(x)"], color = "blue", label = "F(x)", linewidth = 0.5, alpha =0.7)
            axs[num//2, num%2].plot(df_put['x'], df_put["G2(x)"], color = "orange", label = "G(x)", linewidth = 0.5, alpha =0.7)
            axs[num//2, num%2].set_ylim([0, max(1.05,dP_dQ(mu_tilde, sigma_tilde, T, r, W_T_tilde)  + 0.1)])
            axs[num//2, num%2].set_xlim([0,x_range_put])
            axs[num//2, num%2].set_title('F(x) i G(x) w dla opcji put, $\widetilde{W_T}$ =' + f'{W_T_tilde}', fontsize = 10)
            
plt.subplots_adjust(top=1.5)
plt.show()
            

Na poniższych rysunkach możemy zauważyć pewnw podobieństwo między wykresami dla opcji call z parametrem $\widetilde{W_T}$, a opcją put z parametrem $-\widetilde{W_T}$. Również warto zwrócić uwagę że na ostatnim wykresiw funkcja $G_Q(x)$ jest mniejsza od $F_Q(x)$, wynika to z problemów natury numerycznej.

#### Wpływ parametru $K$ na funkcje F(x) i G(x)

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(15,15))
W_T_tilde = 1 
Rad_Nik = dP_dQ(mu_tilde, sigma_tilde, T, r, W_T_tilde)
vec_K = [80, 80, 100, 100, 120, 120, 140, 140]
for num, K in enumerate(vec_K):
    if(num %2 ==0):
        df_call = pd.DataFrame({ "x" : x_call, "G(x)" : None, "G2(x)" : None, "G2'(x)" : None})
        for i in range(len(x_call)):
            df_call.at[i, "G(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 1, derivative = False)
            df_call.at[i, "G2(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 2, derivative = False)
            #df_call.at[i, "G'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 1, derivative = True)
            #df_call.at[i, "G2'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 2, derivative = True)
            axs[num//2, num%2].plot(df_call['x'], df_call["G(x)"], color = "blue", label = "F(x)", linewidth = 0.5, alpha =0.7)
            axs[num//2, num%2].plot(df_call['x'], df_call["G2(x)"], color = "orange", label = "G(x)", linewidth = 0.5, alpha =0.7)
            axs[num//2, num%2].set_ylim([0, max(1.05,dP_dQ(mu_tilde, sigma_tilde, T, r, W_T_tilde) + 0.1)])
            axs[num//2, num%2].set_xlim([0,x_range_call])
            axs[num//2, num%2].set_title('F(x) i G(x) w dla opcji call, $K$ =' + f'{K}', fontsize = 10)
            
            
    else:
        x_put = np.arange(delta, K, delta)
        df_put = pd.DataFrame({ "x" : x_put, "G(x)" : None, "G2(x)" : None, "G2'(x)" : None})
        for i in range(len(x_put)):
            df_put.at[i, "G(x)"] = G_func(df_put.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "put", target_function = 1, derivative = False)
            df_put.at[i, "G2(x)"] = G_func(df_put.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "put", target_function = 2, derivative = False)
            #df_call.at[i, "G'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 1, derivative = True)
            #df_call.at[i, "G2'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 2, derivative = True)
            axs[num//2, num%2].plot(df_put['x'], df_put["G(x)"], color = "blue", label = "F(x)", linewidth = 0.5, alpha =0.7)
            axs[num//2, num%2].plot(df_put['x'], df_put["G2(x)"], color = "orange", label = "G(x)", linewidth = 0.5, alpha =0.7)
            axs[num//2, num%2].set_ylim([0, max(1.05,dP_dQ(mu_tilde, sigma_tilde, T, r, W_T_tilde)  + 0.1)])
            axs[num//2, num%2].set_xlim([0,140])
            axs[num//2, num%2].set_title('F(x) i G(x) w dla opcji put, $K$ =' + f'{K}', fontsize = 10)
            
plt.subplots_adjust(top=1.5)
plt.show()
            

Zmiana parametru K wpływa tempo zbieżności funkcji $F_Q(x)$ i $G_Q(x)$ do siebie. W przypadku opcji call zwiększenie powoduje szybsze zbieganie, a dla opcji put wolniejsze.

#### Wpływ parametru $\rho$ na funkcje $F_Q(x)$ i $G_Q(x)$

In [ ]:
fig, axs = plt.subplots(7, 2, figsize=(15,15))
K = 100
W_T_tilde = 1 
Rad_Nik = dP_dQ(mu_tilde, sigma_tilde, T, r, W_T_tilde)
vec_rho= [-0.99, -0.99, -0.7, -0.7, -0.2, -0.2, 0, 0, 0.2, 0.2, 0.7, 0.7, 0.99, 0.99]
x_put = np.arange(delta, K, delta)
for num, rho in enumerate(vec_rho): 
    if(num %2 ==0):
        df_call = pd.DataFrame({ "x" : x_call, "G(x)" : None, "G2(x)" : None, "G2'(x)" : None})
        for i in range(len(x_call)):
            df_call.at[i, "G(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 1, derivative = False)
            df_call.at[i, "G2(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 2, derivative = False)
            #df_call.at[i, "G'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 1, derivative = True)
            #df_call.at[i, "G2'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 2, derivative = True)
            axs[num//2, num%2].plot(df_call['x'], df_call["G(x)"], color = "blue", label = "F(x)", linewidth = 0.5, alpha =0.7)
            axs[num//2, num%2].plot(df_call['x'], df_call["G2(x)"], color = "orange", label = "G(x)", linewidth = 0.5, alpha =0.7)
            axs[num//2, num%2].set_ylim([0, max(1.05,dP_dQ(mu_tilde, sigma_tilde, T, r, W_T_tilde) + 0.1)])
            axs[num//2, num%2].set_xlim([0,x_range_call])
            axs[num//2, num%2].set_title('F(x) i G(x) w dla opcji call, rho =' + f'{rho}', fontsize = 10)
            
            
    else:
        df_put = pd.DataFrame({ "x" : x_put, "G(x)" : None, "G2(x)" : None, "G2'(x)" : None})
        for i in range(len(x_put)):
            df_put.at[i, "G(x)"] = G_func(df_put.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "put", target_function = 1, derivative = False)
            df_put.at[i, "G2(x)"] = G_func(df_put.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "put", target_function = 2, derivative = False)
            #df_call.at[i, "G'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 1, derivative = True)
            #df_call.at[i, "G2'(x)"] = G_func(df_call.at[i, 'x'], mu, sigma, mu_tilde, sigma_tilde, r, T, K, S0, W_T_tilde, rho, type = "call", target_function = 2, derivative = True)
            axs[num//2, num%2].plot(df_put['x'], df_put["G(x)"], color = "blue", label = "F(x)", linewidth = 0.5, alpha =0.7)
            axs[num//2, num%2].plot(df_put['x'], df_put["G2(x)"], color = "orange", label = "G(x)", linewidth = 0.5, alpha =0.7)
            axs[num//2, num%2].set_ylim([0, max(1.05,dP_dQ(mu_tilde, sigma_tilde, T, r, W_T_tilde)  + 0.1)])
            axs[num//2, num%2].set_xlim([0,140])
            axs[num//2, num%2].set_title('F(x) i G(x) w dla opcji put, rho =' + f'{rho}', fontsize = 10)            
plt.subplots_adjust(top=2.5)
plt.show()

Tutaj ponownie możemy zauważyć podobieństwo między wykresami. Tym razem między opcją call i parametru $\rho$, a opcją put z parametrem $-\rho$

## Optymalizacja pierwszej funkcji celu

### Parametry wyjściowe

In [ ]:
mu_nt = 0.06
mu_t = 0.06
sigma_nt =  0.3
sigma_t =  0.3
r = 0.05
X0_nt = 100
X0_t = 100
T = 1
K = 100
repeat = 10000
values_per_year = 250

In [ ]:
underlying_t = Underlying(mu_t, sigma_t, r, values_per_year)

### Proste *sanity checki* wyznaczanego przez przypadek skrajny payoffu

In [ ]:
rho = 0.999999
underlying_nt = NonTradedUnderlying(mu_t, sigma_t, underlying_t, rho)
[_, reality_t], [_, reality_nt] = underlying_nt.simulate_together_P(repeat, T)

In [ ]:
call_nt = Vanilla_on_NonTraded(underlying_nt, K, T, True)
put_nt = Vanilla_on_NonTraded(underlying_nt, K, T, False)

In [ ]:
call_vanilla = Vanilla(underlying_t, K, T, True)
put_vanilla = Vanilla(underlying_t, K, T, False)

#### Cena opcji w świecie Blacka Scholesa- porównanie

In [ ]:
price_MC_nontradable = call_nt.get_MC_price(X0_t, X0_nt)
BS_call = call_vanilla.get_price(X0_t)
print(f'Analitycna cena opcji call:{BS_call:.3f}')
print(f'MC cena opcji na niehandlowalne imitującej zwykłego calla:{price_MC_nontradable:.3f}')
assert abs(price_MC_nontradable - BS_call)/BS_call < 0.05

In [ ]:
price_MC_nontradable = put_nt.get_MC_price(X0_t, X0_nt)
BS_put = put_vanilla.get_price(X0_t)
print(f'Analitycna cena opcji put:{BS_put:.3f}')
print(f'MC cena opcji na niehandlowalne imitującej zwykłego puta:{price_MC_nontradable:.3f}')
assert abs(price_MC_nontradable - BS_put)/BS_put < 0.05

Widzimy, że estymacje ceny opcji na aktywo niehandlowalne w przypadku imitującym opcję waniliową są stosunkowo zbliżone. Potencjalny, mniejszy niż 5% ceny właściwej błąd wynikać może z metody symulacji Monte Carlo.

#### Zmodyfikowany payoff do hedgowania połową uzyskanych pieniędzy- porównanie

In [ ]:
call_nt.set_m(BS_call / 2, X0_t, X0_nt)
call_nt_quantile_payoff = call_nt.payoff_special(X0_t * reality_t, X0_nt)
put_nt.set_m(BS_put / 2, X0_t, X0_nt)
put_nt_quantile_payoff = put_nt.payoff_special(X0_t * reality_t, X0_nt)

In [ ]:
f_call = payoff_from_v0(call_vanilla, BS_call / 2, X0_nt)[0]
call_quantile_payoff = f_call(X0_nt * reality_nt)
f_put = payoff_from_v0(put_vanilla, BS_put / 2, X0_nt)[0]
put_quantile_payoff = f_put(X0_nt * reality_nt)

In [ ]:
fig, axs = plt.subplots(2,1, sharex = True, sharey = True)
fig.tight_layout(pad=4.0)
axs[0].scatter((X0_t * reality_t).iloc[:,-1], call_nt_quantile_payoff)
axs[0].set_title('Imitacja kwantylowego hedgingu opcji standardowej call opcją na aktywo niehandlowalne')
axs[1].scatter((X0_nt * reality_nt).iloc[:,-1], call_quantile_payoff)
axs[1].set_title('Kwantylowy hedging opcji standardowej call')
plt.suptitle('Payoff zmodyfikowany do hedgingu z użyciem połowy ceny calla')

Widzimy, że dla opcji call zmodyfikowane ze względu na hedging kwantylowy payoffy są bardzo zbliżone.

In [ ]:
fig, axs = plt.subplots(2,1, sharex = True, sharey = True)
fig.tight_layout(pad=4.0)
axs[0].scatter((X0_t * reality_t).iloc[:,-1], put_nt_quantile_payoff)
axs[0].set_title('Imitacja kwantylowego hedgingu opcji standardowej put opcją na aktywo niehandlowalne')
axs[1].scatter((X0_nt * reality_nt).iloc[:,-1], put_quantile_payoff)
axs[1].set_title('Kwantylowy hedging opcji standardowej put')
plt.suptitle('Payoff zmodyfikowany do hedgingu z użyciem połowy ceny puta')

In [ ]:
put_nt.get_MC_price(X0_t, X0_nt)
put_vanilla.payoff_func = f_put
print(f'Imitacja standardowego kwantylowego hedgingu: cena = {put_nt.get_MC_price(X0_t, X0_nt):.3f}, p-stwo sukcesu = {(1 - (put_nt_quantile_payoff == 0) * ((X0_t * reality_t).iloc[:,-1] >= K)).mean():.3f}')
print(f'Standardowy hedging kwantylowy: cena = {put_vanilla.get_MC_price(X0_t):.3f}, p-stwo sukcesu = {(1 - (put_quantile_payoff == 0) * ((X0_nt * reality_nt).iloc[:,-1] >= K)).mean():.3f}')

Widzimy, że w przypadku opcji put zmodyfikowany payoff dla podejścia z aktywem niehandlowalnym ma inną formę niż w przypadku podejścia standardowego (mimo identycznie dobranych parametrów). Jednocześnie ceny tak skonstruowanych zabezpieczń są bardzo zbliżone- podobne są także empiryczne prawdopodobieństwa sukcesu. Kwestia ta nie została przez nas wyjaśniona- nasuwa się, że konstrukcja zmodyfikowanego payoffu nie musi być jednoznaczna. Alternatywnie podejście z dwoma odcięciami payoffu wyprowadzone dla opcji call w artykule źródłowym nie ma bezpośredniego przełożenia na opcję put.

### Wrażliwość nachylenia stycznej do funkcji $G$, a cena zabezpieczenia skonstruowanego w ten sposób payoffu

Poniższe wykresy przedstawiać będą zależność pomiędzy parametrem **m** użytym do konstrukcji zmodyfikowanego payoffu, a kosztem zabezpieczenia takiego payoffu. Dodatkowym czynnikiem wyróżniajacym będzie współczynnik korelacji aktywa głównego (niehandlowalnego) z pomocniczym aktywem handlowalnym, którego używać mielibyśmy do hedgowania.

In [ ]:
rho_call = dict()
for rho in [-0.999, -0.75, -0.25, 0, 0.25, 0.75, 0.999]:
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
    rho_call[rho] = Vanilla_on_NonTraded(underlying_nt, K, T, True)

In [ ]:
m_s = np.arange(0.00001, 0.02, 0.00001)
plt.figure(figsize=(10,10))
plt.xlabel('m', size = 15)
plt.xlim([np.min(m_s), np.max(m_s)])    
plt.xticks(size = 15)
plt.ylabel('V0', size = 15)
plt.ylim([0, 200])
plt.yticks(size = 15)
for num, rho in enumerate(rho_call):
    prices = []
    for m in tqdm(m_s):
        rho_call[rho].m = m
        V0 = rho_call[rho].get_MC_price(X0_t, X0_nt)
        prices.append(V0)
    plt.plot(m_s, prices, lw = 2, label = f'rho = {rho}')
plt.legend(fontsize = 15)
plt.title(f'Wymagany kapitał początkowy zabezpieczenia opcji call(K = {K}) na aktywo niehandlowalne\nw zależności od przyjętego współczynnika m w podziale\nze wzglęgu na korelację z pomocniczym aktywem handlowalnym',size = 15)


In [ ]:
rho_put = dict()
for rho in [-0.999, -0.75, -0.25, 0, 0.25, 0.75, 0.999]:
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
    rho_put[rho] = Vanilla_on_NonTraded(underlying_nt, K, T, False)

In [ ]:
m_s = np.arange(0.00001, 0.02, 0.00001)
plt.figure(figsize=(10,10))
plt.xlabel('m', size = 15)
plt.xlim([np.min(m_s), np.max(m_s)])    
plt.xticks(size = 15)
plt.ylabel('V0', size = 15)
plt.ylim([0, 100])
plt.yticks(size = 15)
for num, rho in enumerate(rho_put):
    prices = []
    for m in tqdm(m_s):
        rho_put[rho].m = m
        V0 = rho_put[rho].get_MC_price(X0_t, X0_nt)
        prices.append(V0)
    plt.plot(m_s, prices, lw = 2, label = f'rho = {rho}')
plt.legend(fontsize = 15)
plt.title(f'Wymagany kapitał początkowy zabezpieczenia opcji put(K = {K}) na aktywo niehandlowalne\nw zależności od przyjętego współczynnika m w podziale\nze wzglęgu na korelację z pomocniczym aktywem handlowalnym',size = 15)


Intuicyjnie **m** określa w pewnym stopniu porządane prawdopodobieństwo zabezpieczenia- im wyższe nachylenie stycznej tym mniejszy jest payoff opcji, na którą mamy zamiar się zabezpieczyć. Pierwszą narzucającą się obserwacją jest fakt, że im bardziej skrajny jest parametr $\rho$ tym większą stabilnością cechują się ceny opcji- od strony implementacyjnej utrudnia to zatem dokładne przełożenie zadanego kapitału początkowego na wymagany do zabezpieczania zmodyfikowany payoff. Ceny opcji są najwyższe dla $\rho$ zbliżonych do $0$- możemy to tłumaczyć faktem, że zabezpieczając się aktywem handlowalnym mamy coraz mniejszą kontrolę nad aktywem, którego dotyczyć będzie ten faktyczny payoff. Widzimy, że ceny opcji są zbliżone dla przeciwnych sobie korelacji- pewnego rodzaju symetryczność ich payoffów zobaczymy w sekcji poniższej.

### Zmodyfikowane payoffy w zależności od parametru **m**

Ostateczna relacja, na której nam zależy to:
$$V_0 \longrightarrow m \longrightarrow \text{zmodyfikowany payoff} \longrightarrow \text{hedging zmodyfikowanego payoffu}$$
Znamy już zależność z poziomu pierwszej strzałki od lewej. Zobaczmy teraz jak parametr **m** wpływa na zmodyfikowany payoff, który ma zostać zabezpieczony. W związku z tym, że nachylenie stycznej do $G$ jest konceptem dość abstrakcyjnym utożsamiajmy je na bazie wykresów poprzednich z odwrotnością posiadanego kapitału- im większe **m** tym mniejszy kapitał, którym dysponuję.

<font color='red'>** UWAGA: Poniższe wykresy będą miały postać animacji- przed jej puszczeniem należy rozwinąć output komórki, a po zakończeniu animacji go zwinąć. **</font>

In [ ]:
rho_call = dict()
for rho in [-0.999, -0.5, 0, 0.5, 0.999]: #można sobie wybrać inne korelacje lub mniej
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
    rho_call[rho] = Vanilla_on_NonTraded(underlying_nt, K, T, True)

In [ ]:
m = 0.0001
diff = 0.00001
rate = 1.9
booster = 3
V0_prev = len(rho_call) * [np.infty]
i = 0
j = 0
while True:
    j += 1
    plt.figure(figsize=(12,8))
    if j == 50:
        diff *= booster
    else:
        for num, rho in enumerate(rho_call):
            option = rho_call[rho]
            final_index = round(option.underlying.values_per_year * option.T + 1)
            sims_t = pd.DataFrame(np.ones(500), columns = [0])
            sims_t[1] = np.arange(10, 510) / X0_t
            option.m = m
            payoff = option.payoff_special(X0_t * sims_t, X0_nt)
            V0 = np.exp(-option.underlying.r * option.T) * payoff.mean()
            if abs(V0 - V0_prev[num]) < 0.1:
                i += 1
            V0_prev[num] = V0
            plt.plot((X0_t * sims_t).iloc[:,-1], payoff, label = f'rho = {rho}, V0 = {V0:.2f}', lw = 2)
        plt.title(f'Zabezpieczenie call(K = {K}) na aktywo niehandlowalne w zależności od parametru m\nzmodyfikowany payoff i wymagany kapitał początkowy\nm = {m:.5f}', size = 20)
        plt.xlim([20,500])
        plt.xlabel('Cena końcowa aktywa handlowalnego', size = 12)
        plt.ylim([-5,500])
        plt.ylabel('payoff', size= 12)
        plt.legend(loc = 'upper right', prop={'size': 12})
    m += diff
    if i == len(rho_call):
        i = 0
        diff *= rate
    display.clear_output(wait = True)
    display.display(plt.gcf())
    sleep(0.05)
    if np.max(np.array(V0_prev)) < 1:
        break

Widzimy, że dla $\rho$ zbliżonego do $1$ payoff zmodyfikowany przypomina ten standardowy dla kwantylowo hedgowanej opcji call. W przypadku przeciwnym ma on postać jak dla opcji put- jeśli aktywa zachowują się skrajnie przeciwnie musimy zabezpieczać się na spadki aktywa handlowalnego oczekując tym samym wzrostów aktywa głównego. Kolejną obserwacją jest to, że dla mniejszej, dodatniej korelacji payoff staje się wklęsły oraz niezerowy także dla cen poniżej $K$. Intuicyjnie mamy tu do czynienia z pewnego rodzaju niepewnością faktycznego ruchu aktywa głównego, co skutkuje w większej "konserwatywności" wymaganych końcowych stanów portfela. Podobny trend obserwujemy także dla korelacji ujemnej- tym razem wykres jest jednak wypukły. Wklęsłość tą można tłumaczyć faktem, że o ile spadek aktywa handlowalnego jest ograniczony z dołu tak w tym wypadku potencjalny wzrost aktywa głównego nie ma żadnego ograniczenia. Przypadek, gdzie aktywa są nieskorelowane jest właściwie zabezpieczaniem się na szum. Odpowiada wtedy właściwie dosłownie zakumulowanemu kapitałowi początkowemu- im więcej go mamy tym na wyższy kwantyl losowości jesteśmy zabezpieczeni. W tym wypadku hedging polegałby raczej na trzymaniu odpowiedniej kwoty bez konieczności handlowania aktywem pomocniczym.

In [ ]:
rho_put = dict()
for rho in [-0.999, -0.5, 0, 0.5, 0.999]: #można sobie wybrać inne korelacje lub mniej
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
    rho_put[rho] = Vanilla_on_NonTraded(underlying_nt, K, T, False)

In [ ]:
m = 0.0001
diff = 0.00002
rate = 1.9
V0_prev = len(rho_put) * [np.infty]
i = 0
j = 0
while True:
    j += 1
    plt.figure(figsize=(12,8))
    if j == 50:
        #img = mpimg.imread('xd.jpeg')
        #plt.imshow(img)
        diff *= 2
    for num, rho in enumerate(rho_put):
        option = rho_put[rho]
        final_index = round(option.underlying.values_per_year * option.T + 1)
        _, sims_full_t = option.MC_setup[0]
        sims_t = pd.DataFrame(np.ones(500), columns = [0])
        sims_t[1] = np.arange(10, 510) / X0_t
        option.m = m
        payoff = option.payoff_special(X0_t * sims_t, X0_nt)
        V0 = np.exp(-option.underlying.r * option.T) * payoff.mean()
        if abs(V0 - V0_prev[num]) < 0.1:
            i += 1
        V0_prev[num] = V0
        plt.plot((X0_t * sims_t).iloc[:,-1], payoff, label = f'rho = {rho}, V0 = {V0:.2f}', lw = 2)
    plt.title(f'Zabezpieczenie put(K = {K}) na aktywo niehandlowalne w zależności od parametru m\nzmodyfikowany payoff i wymagany kapitał początkowy\nm = {m:.5f}', size = 20)
    plt.xlim([20,500])
    plt.xlabel('Cena końcowa aktywa handlowalnego', size = 12)
    plt.ylim([-5,100])
    plt.ylabel('payoff', size= 12)
    plt.legend(loc = 'upper right', prop={'size': 12})
    m += diff
    if i == 5:
        i = 0
        diff *= rate
    display.clear_output(wait = True)
    display.display(plt.gcf())
    sleep(0.15)
    if np.max(np.array(V0_prev)) < 1:
        break

Podobne zachowanie możemy zaobserwować dla opcji put. Warto odnotować, że w tym wypadku payoff zmodyfikowany nie może przekroczyć ceny wykonania $K$- jest to maksymalna wypłata payoffu faktycznego. Warto zauważyć, że dla dużych **m** payoffy (poza przypadkiem bez korelacji) zaczynają zbiegać do punktów skupienia z największą masą aktywa niehandlowalnego- przykładowo dla korelacji zbliżonej do $1$ są to okolice ceny początkowej tego aktywa. Trend ten widoczny był także dla opcji kupna.

#### Rezultaty $\Delta$-hedgingu opcji na aktywo niehandlowalne

Ostatnim elementem zaprezentowanego wcześniej ciągu zależności było skonstruowanie strategii zabezpieczającej opcję przy zadanym kapitale początkowym. Teraz przyjrzyjmy się zatem jakie wyniki osiągają tak skonstruowane strategie.

Rozpocznijmy od opcji call i kapitału początkowego w kwocie połowy ceny opcji standardowej (o handlowalnym aktywie bazowym). Oczekujemy zatem, że dla $\rho$ zbliżonego do $1$ zabezpieczenie będzie najbliższe pełnemu, a zmodyfikowany payoff dla aktywa handlowalnego odpowiadać będzie zabezpieczonym przypadkom aktywa głównego. Dla wszystkich pozostałych ze względu na dodatkową niepewność (oraz w oparciu o wcześniej widziane wykresy) odpowiednio zahedgowaych przypadków powinno być mniej.

In [ ]:
repeat = 1000

In [ ]:
fig, axs = plt.subplots(3,5, figsize = (25, 15), sharey = 'row')
fig.tight_layout(pad=6.0)
V0 = BS_call / 2
for idx, rho in tqdm(enumerate([0.999999, 0.9, 0.75, 0.5, 0.25])):
    underlying_nt = NonTradedUnderlying(mu_t, sigma_t, underlying_t, rho)
    [_, reality_t], [_, reality_nt] = underlying_nt.simulate_together_P(repeat, T)
    
    call_nt = Vanilla_on_NonTraded(underlying_nt, K, T, True)
    call_nt.set_m(V0, X0_t, X0_nt)
    sims_t = pd.DataFrame(np.ones(150), columns = [0])
    sims_t[1] = np.arange(50, 200) / X0_t
    payoff = call_nt.payoff_special((X0_t * sims_t), X0_nt)

    money_time_call = pd.DataFrame(np.zeros(reality_t.shape))
    delta_time_call = pd.DataFrame(np.zeros(reality_t.shape))
    with Pool(processors) as p:
            results = p.map(lambda i: trader_loop(i, call_nt, V0, (X0_t*reality_t), (X0_nt*reality_nt)) , np.arange(0,repeat))
    for num, item in enumerate(results):
        money_time_call.loc[num] = item[0]
        delta_time_call.loc[num] = item[1]   
    money_time_call_outcome = money_time_call.copy()
    money_time_call_outcome['outcome'] = money_time_call.apply(lambda row: 'success' if row.iloc[-1] >= - V0 else 'fail', axis = 1)
    sb.histplot(ax = axs[0, idx], x = money_time_call_outcome[underlying_nt.values_per_year * T], bins = repeat // 20, hue = money_time_call_outcome['outcome'], palette = {'fail':'tab:blue','success':'tab:orange'}, legend = True, stat = 'percent')
    axs[0, idx].set_xlim([-150, 50])        
    axs[0, idx].set_title(f'rho = {rho}\n\nKońcowy stan portfela\np-stwo sukcesu = {(money_time_call_outcome["outcome"] == "success").mean():.3f}', fontsize = 15)
    axs[0, idx].set_xlabel('Końcowy stan portfela', fontsize = 12)
    axs[1, idx].plot((X0_t * sims_t).iloc[:,-1], payoff, lw = 3)
    axs[1, idx].set_title('Zmodyfikowany payoff dla aktywa handlowalnego', fontsize = 13)
    axs[1, idx].set_xlim([50,200])    
    axs[1, idx].set_xlabel('Końcowa cena aktywa handlowalnego')
    sb.scatterplot(ax = axs[2, idx], x = (X0_nt*reality_nt).iloc[:,-1], y = np.maximum((X0_nt*reality_nt)[underlying_nt.values_per_year * T] - K,0), hue = money_time_call_outcome['outcome'], palette = {'fail':'tab:blue','success':'tab:orange'}).set_title('Ostateczny payoff opcji', fontsize = 15)
    axs[2, idx].set_xlim([50,200])    
    axs[2, idx].set_xlabel('Końcowa cena aktywa niehandlowalnego', fontsize = 12)
    if idx == 0:
        axs[1, idx].set_ylabel('Payoff zmodyfikowany', fontsize = 12)    
        axs[2, idx].set_ylabel('Payoff faktyczny', fontsize = 12)    
plt.suptitle(f'Quantile hedging call(K = {K}) na aktywo niehandlowalne przy V0 = {V0:.2f} dla różnych korelacji z pomocniczym aktywem handlowalnym', fontsize= 18)
plt.subplots_adjust(top=0.9)


W związku z tym, że jest to pierwsze ukazanie tej serii wykresów zacznijmy od słowa wstępu co one faktycznie przedstawiają. Każda kolumna odpowiada opcji call na aktywo niehandlowalne zabezpieczanej aktywem handlowalnym, które jest skorelowane z określonym $\rho$. Pierwszy wiersz wykresów skupia się na ostatecznym wyniku portfeli przeprowadzających proces hedgingu. Wyniki te zaprezentowane są w postaci histogramu. Rząd pierwszy oparty jest o symulacje rzeczywistości- w konsekwencji otrzymane wyniki dzielimy na sukcesy oraz porażki. Podział ten jest tutaj dość liberalny, za sukces uznajemy bowiem wynik portfela nieniższy niż $-10$ zł. Drugi rząd skupia się na ukazaniu payoffu zmodyfikowanego dla aktywa handlowalnego, którego zabezpieczanie ma niejako zastąpić bezpośrednie hedgowanie aktywa głównego. Wiersz ostatni ukazuje nam natomiast faktyczny payoff opcji call w zależności od ceny końcowej aktywa niehandlowalnego (głównego). Wykresy te stworzone są za pomocą faktycznych rezultatów symulacji- w konsekwencji kolorystycznie rozdzielamy je w zgodzie z metodyką z wiersza pierwszego. Frakcja sukcesów reprezentuje nam tutaj także empiryczne prawdopodobieństwo sukcesu umieszczone na górze każdej kolumny.

Przejdźmy teraz do opisu faktycznych otrzymanych wyników dla opcji call i korelacji dodatnich.

Zgodnie z wcześniejszą intuicją widzimy, że w przypadku korelacji bardzo zbliżonej do $1$ (kolumna pierwsza) frakcja sukcesów jest zdecydowanie największa (a w konsekwencji również empiryczne prawdopodobieństwo sukcesu). Niezerowa część payoffu zmodyfikowanego faktycznie odpowiada z pewną niedokładnością zabezpieczonej części payoffów faktycznych (nadal w pierwszej kolumnie- wiersz drugi oraz trzeci). W przypadku mniejszych korelacji prawdopodobieństwa sukcesu maleją- wykresy zaprezentowane wcześniej pokazały nam, że mniejsze korelacje cechują się większym wymaganym kapitałem do zabezpieczenia opcji na określonym poziomie. Same zmodyfikowane payoffy coraz bardziej wypłaszczają się niezerując się jednocześnie na coraz większym przedziale. Wypłaszczenie ma związek oczywiście z niezmieniającą się liczbą kapitału- większy przedział całkowity oznacza mniejsze zabezpieczenie na każdym z podprzedziałów. Wraz ze wzrostem losowości aktywa niehandlowalnego w stosunku do aktywa handlowalnego spada zakres zabezpieczonych ostatecznie payoffów- dla $\rho=0.25$ ogranicza się on właściwie tylko do payoffów zerowych.

In [ ]:
fig, axs = plt.subplots(3,4, figsize = (20, 15), sharey = 'row')
fig.tight_layout(pad=6.0)
V0 = BS_call / 2
for idx, rho in tqdm(enumerate([-0.999999, -0.75, -0.25, 0])):
    underlying_nt = NonTradedUnderlying(mu_t, sigma_t, underlying_t, rho)
    [_, reality_t], [_, reality_nt] = underlying_nt.simulate_together_P(repeat, T)
    
    call_nt = Vanilla_on_NonTraded(underlying_nt, K, T, True)
    call_nt.set_m(V0, X0_t, X0_nt)
    sims_t = pd.DataFrame(np.ones(150), columns = [0])
    sims_t[1] = np.arange(50, 200) / X0_t
    payoff = call_nt.payoff_special((X0_t * sims_t), X0_nt)

    money_time_call = pd.DataFrame(np.zeros(reality_t.shape))
    delta_time_call = pd.DataFrame(np.zeros(reality_t.shape))
    with Pool(processors) as p:
            results = p.map(lambda i: trader_loop(i, call_nt, V0, (X0_t*reality_t), (X0_nt*reality_nt)) , np.arange(0,repeat))
    for num, item in enumerate(results):
        money_time_call.loc[num] = item[0]
        delta_time_call.loc[num] = item[1]   
    money_time_call_outcome = money_time_call.copy()
    money_time_call_outcome['outcome'] = money_time_call.apply(lambda row: 'success' if row.iloc[-1] >= - 10 else 'fail', axis = 1)
    sb.histplot(ax = axs[0, idx], x = money_time_call_outcome[underlying_nt.values_per_year * T], bins = repeat // 20, hue = money_time_call_outcome['outcome'], palette = {'fail':'tab:blue','success':'tab:orange'}, legend = True, stat = 'percent')
    axs[0, idx].set_xlim([-150, 50])        
    axs[0, idx].set_title(f'rho = {rho}\n\nKońcowy stan portfela\np-stwo sukcesu = {(money_time_call_outcome["outcome"] == "success").mean():.3f}', fontsize = 15)
    axs[0, idx].set_xlabel('Końcowy stan portfela', fontsize = 12)
    axs[1, idx].plot((X0_t * sims_t).iloc[:,-1], payoff, lw = 3)
    axs[1, idx].set_title('Zmodyfikowany payoff dla aktywa handlowalnego', fontsize = 13)
    axs[1, idx].set_xlim([50,200])    
    axs[1, idx].set_xlabel('Końcowa cena aktywa handlowalnego')
    sb.scatterplot(ax = axs[2, idx], x = (X0_nt*reality_nt).iloc[:,-1], y = np.maximum((X0_nt*reality_nt)[underlying_nt.values_per_year * T] - K,0), hue = money_time_call_outcome['outcome'], palette = {'fail':'tab:blue','success':'tab:orange'}).set_title('Ostateczny payoff opcji', fontsize = 15)
    axs[2, idx].set_xlim([50,200])    
    axs[2, idx].set_xlabel('Końcowa cena aktywa niehandlowalnego', fontsize = 12)
    if idx == 0:
        axs[1, idx].set_ylabel('Payoff zmodyfikowany', fontsize = 12)    
        axs[2, idx].set_ylabel('Payoff faktyczny', fontsize = 12)    
plt.suptitle(f'Quantile hedging call(K = {K}) na aktywo niehandlowalne przy V0 = {V0:.2f} dla różnych korelacji z pomocniczym aktywem handlowalnym', fontsize= 18)
plt.subplots_adjust(top=0.89)


Na zamieszczonych wykresach niejednoznaczna okazuje się relacja pomiędzy frakcją sukcesów, a wielkością korelacji. Mimo, że dla $\rho=-0.75$ p-stwo zabezpieczenia było niższe niż w przypadku skrajnym to już w dla korelacji na poziomie $-0.25$ wynik okazał się lepszy. Dodatkowo uwzględniony tu przypadek nieskorelowania aktywów otrzymał co ciekawe najlepsze prawdopodobieńswo sukcesu. Może to wynikać z faktu, że portfel w czasie jest tutaj dużo bardziej stabilny- skoro nie mamy żadnej kontroli nad aktywem głównym nie mamy potrzeby wykonywać transakcji na tym handlowalnym. W kontekście zmodyfikowanych payoffów widzimy analogiczny trend jak dla korelacji dodatnich choć oczywiście wykresy są tutaj odwrócone. Zauważmy, że jeśli korelacja jest ujemna zabezpieczamy niejako nielimitowany wzrost aktywa niehandlowalnego tracącym jednocześnie na wartości aktywem handlowalnym- pierwsze z aktywów jest zatem dużo bardziej zmienne niż drugie, które od dołu ogranicza nieosiągalna wartość $0$. Ta dysproporcja może stanowić o stosunkowo słabych wynikach hedgingów, które zabezpieczają głównie przypadki, gdzie do payoffu nie doszło.

Rozważmy teraz opcję put na aktywo niehandlowalne o tych samych parametrach, co analizowany wyżej call oraz z kapitałem początkowym równym połowie ceny jaką zapłacilibyśmy za jej waniliowy odpowiednik.

In [ ]:
fig, axs = plt.subplots(3,5, figsize = (25, 15), sharey = 'row')
fig.tight_layout(pad=6.0)
V0 = BS_put / 2
for idx, rho in tqdm(enumerate([0.999999, 0.9, 0.75, 0.5, 0.25])):
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
    [_, reality_t], [_, reality_nt] = underlying_nt.simulate_together_P(repeat, T)
    
    put_nt = Vanilla_on_NonTraded(underlying_nt, K, T, False)
    put_nt.set_m(V0, X0_t, X0_nt)
    sims_t = pd.DataFrame(np.ones(150), columns = [0])
    sims_t[1] = np.arange(50, 200) / X0_t
    payoff = put_nt.payoff_special((X0_t * sims_t), X0_nt)

    money_time_put = pd.DataFrame(np.zeros(reality_t.shape))
    delta_time_put = pd.DataFrame(np.zeros(reality_t.shape))
    with Pool(processors) as p:
            results = p.map(lambda i: trader_loop(i, put_nt, V0, (X0_t*reality_t), (X0_nt*reality_nt)) , np.arange(0,repeat))
    for num, item in enumerate(results):
        money_time_put.loc[num] = item[0]
        delta_time_put.loc[num] = item[1]   
    money_time_put_outcome = money_time_put.copy()
    money_time_put_outcome['outcome'] = money_time_put.apply(lambda row: 'success' if row.iloc[-1] >= - 10 else 'fail', axis = 1)
    sb.histplot(ax = axs[0, idx], x = money_time_put_outcome[underlying_nt.values_per_year * T], bins = repeat // 20, hue = money_time_put_outcome['outcome'], palette = {'fail':'tab:blue','success':'tab:orange'}, legend = True, stat = 'percent')
    axs[0, idx].set_xlim([-150, 50])        
    axs[0, idx].set_title(f'rho = {rho}\n\nKońcowy stan portfela\np-stwo sukcesu = {(money_time_put_outcome["outcome"] == "success").mean():.3f}', fontsize = 15)
    axs[0, idx].set_xlabel('Końcowy stan portfela', fontsize = 12)
    axs[1, idx].plot((X0_t * sims_t).iloc[:,-1], payoff, lw = 3)
    axs[1, idx].set_title('Zmodyfikowany payoff dla aktywa handlowalnego', fontsize = 13)
    axs[1, idx].set_xlim([50,200])    
    axs[1, idx].set_xlabel('Końcowa cena aktywa handlowalnego')
    sb.scatterplot(ax = axs[2, idx], x = (X0_nt*reality_nt).iloc[:,-1], y = np.maximum(K - (X0_nt*reality_nt)[underlying_nt.values_per_year * T],0), hue = money_time_put_outcome['outcome'], palette = {'fail':'tab:blue','success':'tab:orange'}).set_title('Ostateczny payoff opcji', fontsize = 15)
    axs[2, idx].set_xlim([50,200])    
    axs[2, idx].set_xlabel('Końcowa cena aktywa niehandlowalnego', fontsize = 12)
    if idx == 0:
        axs[1, idx].set_ylabel('Payoff zmodyfikowany', fontsize = 12)    
        axs[2, idx].set_ylabel('Payoff faktyczny', fontsize = 12)    
plt.suptitle(f'Quantile hedging put(K = {K}) na aktywo niehandlowalne przy V0 = {V0:.2f} dla różnych korelacji z pomocniczym aktywem handlowalnym', fontsize= 18)
plt.subplots_adjust(top=0.9)


Tak jak w przypadku opcji call widzimy, że dla korelacji dodatnich mniejsza korelacja oznacza przy zadanym kapitale początkowym gorszy procent symulacji zabezpieczonych. W przypadku korelacji praktycznie idealnej widzimy, że zmodyfikowany payoff dla aktywa handlowalnego dobrze oddaje kwantylowy hedging opcji na aktywo handlowalne. Im mniejsze $\rho$ tym payoff dla używanego aktywa handlowalnego skupia się (pozostając niezerowym) na coraz mniejszym i bardziej przesuniętym w lewo przedziale. Oddalanie się prawego krańca od ceny wykonania możemy tłumaczyć tym, że jeśli korelacja jest mniejsza to bardziej prawdopodobne jest, że przy malejącej cenie aktywa handlowalnego, aktywo niehandlowalne urośnie, a więc nie poskutkuje żadnym payoffem. Nie mamy tu zatem zachowania znanego nam z opcji call, payoff nie może wypłaszczać się niejako ku coraz mniejszym cenom, ponieważ z dołu ogranicza go wartość $0$. W konsekwencji staje się on dla opcji put coraz "wyższy" kumulując coraz większy kapitał w zabezpieczenie coraz węższego obszaru. Tendencja ta nie do końca sprawdza się dla najmniejszej z rozważanych tu korelacji, gdzie niezerowy fragment ponownie się poszerza- może to mieć związek z ograniczeniem górnym wartości samego payoffu równym cenie wykonania. Jeśli zaś chodzi o faktyczne payoffy, na które okazaliśmy się zabezpieczeni widzimy, że im mniejsza korelacja między aktywami tym potencjalne zabezpieczenie staje się coraz bardziej losowe. Portfel nie jest idealnie skalibrowany pod aktywo, które decyduje o wypłacie opcji- potencjalne sukcesy wynikają bardziej z ułożenia cen dwóch instrumentów.

In [ ]:
fig, axs = plt.subplots(3,4, figsize = (20, 15), sharey = 'row')
fig.tight_layout(pad=6.0)
V0 = BS_put / 2
for idx, rho in tqdm(enumerate([-0.999999, -0.75, -0.25, 0])):
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
    [_, reality_t], [_, reality_nt] = underlying_nt.simulate_together_P(repeat, T)
    
    put_nt = Vanilla_on_NonTraded(underlying_nt, K, T, False)
    put_nt.set_m(V0, X0_t, X0_nt)
    sims_t = pd.DataFrame(np.ones(150), columns = [0])
    sims_t[1] = np.arange(50, 200) / X0_t
    payoff = put_nt.payoff_special((X0_t * sims_t), X0_nt)

    money_time_put = pd.DataFrame(np.zeros(reality_t.shape))
    delta_time_put = pd.DataFrame(np.zeros(reality_t.shape))
    with Pool(processors) as p:
            results = p.map(lambda i: trader_loop(i, put_nt, V0, (X0_t*reality_t), (X0_nt*reality_nt)) , np.arange(0,repeat))
    for num, item in enumerate(results):
        money_time_put.loc[num] = item[0]
        delta_time_put.loc[num] = item[1]   
    money_time_put_outcome = money_time_put.copy()
    money_time_put_outcome['outcome'] = money_time_put.apply(lambda row: 'success' if row.iloc[-1] >= - 10 else 'fail', axis = 1)
    sb.histplot(ax = axs[0, idx], x = money_time_put_outcome[underlying_nt.values_per_year * T], bins = repeat // 20, hue = money_time_put_outcome['outcome'], palette = {'fail':'tab:blue','success':'tab:orange'}, legend = True, stat = 'percent')
    axs[0, idx].set_xlim([-150, 50])        
    axs[0, idx].set_title(f'rho = {rho}\n\nKońcowy stan portfela\np-stwo sukcesu = {(money_time_put_outcome["outcome"] == "success").mean():.3f}', fontsize = 15)
    axs[0, idx].set_xlabel('Końcowy stan portfela', fontsize = 12)
    axs[1, idx].plot((X0_t * sims_t).iloc[:,-1], payoff, lw = 3)
    axs[1, idx].set_title('Zmodyfikowany payoff dla aktywa handlowalnego', fontsize = 13)
    axs[1, idx].set_xlim([50,200])    
    axs[1, idx].set_xlabel('Końcowa cena aktywa handlowalnego')
    sb.scatterplot(ax = axs[2, idx], x = (X0_nt*reality_nt).iloc[:,-1], y = np.maximum(K - (X0_nt*reality_nt)[underlying_nt.values_per_year * T],0), hue = money_time_put_outcome['outcome'], palette = {'fail':'tab:blue','success':'tab:orange'}).set_title('Ostateczny payoff opcji', fontsize = 15)
    axs[2, idx].set_xlim([50,200])    
    axs[2, idx].set_xlabel('Końcowa cena aktywa niehandlowalnego', fontsize = 12)
    if idx == 0:
        axs[1, idx].set_ylabel('Payoff zmodyfikowany', fontsize = 12)    
        axs[2, idx].set_ylabel('Payoff faktyczny', fontsize = 12)    
plt.suptitle(f'Quantile hedging put(K = {K}) na aktywo niehandlowalne przy V0 = {V0:.2f} dla różnych korelacji z pomocniczym aktywem handlowalnym', fontsize= 18)
plt.subplots_adjust(top=0.89)


Kiedy korelacje są ujemne zabezpieczenia stają się gorsze (pod kątem frakcji symulacji zabepieczonych). Wyraźny trend widoczny jest w kontekście maksymalnej straty portfela, rośnie ona wraz ze zbliżaniem się korelacji do $0$- w tym właśnie przypadku sięga ona ponad $-150$. Wynika to z faktu, że zerowa korelacja oznacza całkowity brak kontroli na zachowanie aktywa niehandlowalnego- w tym wypadku także ostateczne payoffy są zabezpieczane w sposób częściowo losowy (pojedyncze pomarańczowe kropki na całym niezerowym payoffie). Wspomniane już prawdopodobieństwo sukcesu nieznacznie różni się pomiędzy przypadkami różnych $\rho$- różnią się natomiast formy payoffu zmodyfikowanego. Payoff ten rośnie wzrast ze wzrostem końcowej ceny aktywa pomocniczego, a więc jego zachowanie jest przeciwne do payoffu faktycznego. Ma to oczywiście związek ze znakiem korelacji- aktywa powinny zachowywac się w sposób sobie częściowo przeciwny.

### Stan portfela w czasie- jak zmienia się ilość posiadanego aktywa handlowalnego w czasie?

Wiemy już, że hedging próbuje posiadanym aktywem handlowalnym niejako dopasowywać się do charakterystyki aktywa niehandlowalnego. Na poniższych wykresach na przykładzie trzech skrajnych symulacji aktywa niehandlowalnego zobaczymy jak zachowywało się aktywo handlowalne oraz zmianę jego ilości w zabezpieczającym portfelu. Kapitał początkowy wynosić będzie ponownie połowę ceny waniliowego odpowiednika rozważanej opcji. W konsekwencji każdy z hedgingów będzie hedgingiem kwantylowym nie gwarantującym pełnego zabezpieczenia. Z tego powodu w pierwszym wierszu ukażemy postać zmodyfikowanego payoffu, który kluczowy jest w ilości posiadanego w portfelu zabezpieczającym aktywa handlowalnego. Czynnikiem zmieniającym się względem każdej z kolumn będzie korelacja $\rho$- zaprezentujemy przypadek idealnego skorelowania dodatniego, ujemnego oraz braku korelacji.

Rozpocznijmy rozważania od opcji kupna.

In [ ]:
repeat = 100

In [ ]:
V0 = BS_call / 2
fig, axs = plt.subplots(4,3, figsize = (20, 15), sharey = 'row')
fig.tight_layout(pad=7.0)
plt.subplots_adjust(top=0.9)
for idx, rho in enumerate([0.9999, 0, -0.9999]):
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
    [_, reality_t], [_, reality_nt] = underlying_nt.simulate_together_P(repeat, T)
    call_nt = Vanilla_on_NonTraded(underlying_nt, K, T, True)
    call_nt.set_m(V0, X0_t, X0_nt)   
    sims_t = pd.DataFrame(np.ones(250), columns = [0])
    sims_t[1] = np.arange(10, 260) / X0_t
    payoff = call_nt.payoff_special((X0_t * sims_t), X0_nt)
    
    money_time_call = pd.DataFrame(np.zeros(reality_t.shape))
    delta_time_call = pd.DataFrame(np.zeros(reality_t.shape))
    with Pool(processors) as p:
            results = p.map(lambda i: trader_loop(i, call_nt, V0, (X0_t*reality_t), (X0_nt*reality_nt)) , np.arange(0,repeat))
    for num, item in enumerate(results):
        money_time_call.loc[num] = item[0]
        delta_time_call.loc[num] = item[1]   
    axs[0, idx].plot((X0_t * sims_t).iloc[:,-1], payoff, lw = 3)
    axs[0, idx].set_xlim([20, 250])
    axs[0, idx].set_ylim([0, 100])
    axs[0, idx].set_xlabel('Końcowa cena aktywa handlowalnego')
    axs[0, idx].set_ylabel('Payoff', size = 15)
    axs[0, idx].set_title(f'rho = {rho}\n\nZmodyfikowany payoff do zabezpieczenia', size = 15)
    indices = [(X0_nt * reality_nt).iloc[:,-1].idxmax(),(X0_nt * reality_nt).iloc[:,-1].idxmin(), abs((X0_nt * reality_nt).iloc[:,-1] - K).idxmin()]
    (X0_nt*reality_nt.iloc[indices,:]).T.plot.line(legend = False, ax = axs[1, idx], lw=2)
    axs[1, idx].set_xlim([0,values_per_year])
    axs[1, idx].set_xlabel('t')
    axs[1, idx].set_ylabel('Cena aktywa niehandlowalnego', size = 15)
    axs[1, idx].set_title('Ceny niehandlowalnego aktywa bazowego w czasie', size = 15)
    axs[1, idx].axhline(y=K, xmin=0, xmax=249, color='black', linestyle='--', lw=2)
    (X0_t*reality_t.iloc[indices,:]).T.plot.line(legend = False, ax = axs[2, idx], lw=2)
    axs[2, idx].set_xlim([0,values_per_year])
    axs[2, idx].set_xlabel('t')
    axs[2, idx].set_ylabel('Cena aktywa handlowalnego', size = 15)
    axs[2, idx].set_title('Ceny handlowalnego aktywa bazowego w czasie', size = 15)
    (delta_time_call.iloc[indices,:-1]).T.plot(legend = False, ax = axs[3, idx], lw=2)
    axs[3, idx].set_xlim([0,values_per_year])
    axs[3, idx].set_xlabel('t')
    axs[3, idx].set_ylabel('Posiadane aktywo handlowalne', size = 15)
    axs[3, idx].set_title('Ilość aktywa handlowalnego w czasie', size = 15)
    axs[3, idx].set_ylim([-1.25, 1.25])
plt.suptitle(f'Hedging kwantylowy calla(K={K}) na aktywo niehandlowalne dla skrajnych korelacji z pomocniczym aktywem handlowalnym', size = 20)

W przypadku opcji kupna i korelacji skrajnie dodatniej widzimy, że trend obu aktywów jest podobny. Zacznijmy od analizy stanu portfela w przypadku symulacji pomarańczowych- aktywo niehandlowalne (jak również handlowalne) szybko spada poniżej cenę wykonania, co zeruje ilość posiadanego instrumentu. Symulacje niebieskie, a więc te z najwyższymi cenami w przypadku wystarczającego kapitału na pełne zabezpieczenie powinny skutkować zbliżającą się ku $1$ wartością parametru $\Delta$. Zauważmy jednak, że ze względu na kapitał ograniczony payoff, na który się przygotowujemy zeruje się powyżej $140$- w konsekwencji zeruje się także ilość aktywa bazowego służącego do zabezpieczania. Ostatnia z par symulacji- te układające się blisko ceny wykonania skutkują stosunkowo wysoką wartością posiadanej $\Delta$ ze względu na dużą szansę, że do dojdzie do "wykonania" dla payoffu zmodyfikowanego. W przypadku korelacji skrajnie ujemnej wnioski są stosunkowo podobne choć oczywiście znak $\rho$ obraca nam "wymogi" na aktywie handlowalnym. Warto tu odnotować dodatkowy wniosek- rezultaty portfela mogą być w tym wypadku gorsze niż dla korelacji dodatniej ze względu na nieintuicyjne zachowanie kupowania intensywnie aktywa, którego wartość spada. Dla korelacji zerowej zachowanie ilości aktywa bazowego w portfelu jest najbardziej losowe ze względu na brak żadnej kontroli aktywa głównego. Próbujemy tutaj niejako zabezpieczać szum, skutkuje to dużą niepewnością i $\Delta$, która nigdy nie przekracza co do wartości bezwzględnej $0.5$.

In [ ]:
V0 = BS_put / 2
fig, axs = plt.subplots(4,3, figsize = (20, 15), sharey = 'row')
fig.tight_layout(pad=7.0)
plt.subplots_adjust(top=0.9)
for idx, rho in enumerate([0.9999, 0, -0.9999]):
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
    [_, reality_t], [_, reality_nt] = underlying_nt.simulate_together_P(repeat, T)
    put_nt = Vanilla_on_NonTraded(underlying_nt, K, T, False)
    put_nt.set_m(V0, X0_t, X0_nt)    
    sims_t = pd.DataFrame(np.ones(250), columns = [0])
    sims_t[1] = np.arange(10, 260) / X0_t
    payoff = put_nt.payoff_special((X0_t * sims_t), X0_nt)
    
    money_time_put = pd.DataFrame(np.zeros(reality_t.shape))
    delta_time_put = pd.DataFrame(np.zeros(reality_t.shape))
    with Pool(processors) as p:
            results = p.map(lambda i: trader_loop(i, put_nt, V0, (X0_t*reality_t), (X0_nt*reality_nt)) , np.arange(0,repeat))
    for num, item in enumerate(results):
        money_time_put.loc[num] = item[0]
        delta_time_put.loc[num] = item[1]   
    axs[0, idx].plot((X0_t * sims_t).iloc[:,-1], payoff, lw = 3)
    axs[0, idx].set_xlim([20, 250])
    axs[0, idx].set_ylim([0, 100])
    axs[0, idx].set_xlabel('Końcowa cena aktywa handlowalnego')
    axs[0, idx].set_ylabel('Payoff', size = 15)
    axs[0, idx].set_title(f'rho = {rho}\n\nZmodyfikowany payoff do zabezpieczenia', size = 15)
    indices = [(X0_nt * reality_nt).iloc[:,-1].idxmax(),(X0_nt * reality_nt).iloc[:,-1].idxmin(), abs((X0_nt * reality_nt).iloc[:,-1] - K).idxmin()]
    (X0_nt*reality_nt.iloc[indices,:]).T.plot.line(legend = False, ax = axs[1, idx], lw=2)
    axs[1, idx].set_xlim([0,values_per_year])
    axs[1, idx].set_xlabel('t')
    axs[1, idx].set_ylabel('Cena aktywa niehandlowalnego', size = 15)
    axs[1, idx].set_title('Ceny niehandlowalnego aktywa bazowego w czasie', size = 15)
    axs[1, idx].axhline(y=K, xmin=0, xmax=249, color='black', linestyle='--', lw=2)
    (X0_t*reality_t.iloc[indices,:]).T.plot.line(legend = False, ax = axs[2, idx], lw=2)
    axs[2, idx].set_xlim([0,values_per_year])
    axs[2, idx].set_xlabel('t')
    axs[2, idx].set_ylabel('Cena aktywa handlowalnego', size = 15)
    axs[2, idx].set_title('Ceny handlowalnego aktywa bazowego w czasie', size = 15)
    (delta_time_put.iloc[indices,:-1]).T.plot(legend = False, ax = axs[3, idx], lw=2)
    axs[3, idx].set_xlim([0,values_per_year])
    axs[3, idx].set_xlabel('t')
    axs[3, idx].set_ylabel('Posiadane aktywo handlowalne', size = 15)
    axs[3, idx].set_title('Ilość aktywa handlowalnego w czasie', size = 15)
    axs[3, idx].set_ylim([-1.25,1.25])
plt.suptitle(f'Hedging kwantylowy puta(K={K}) na aktywo niehandlowalne dla skrajnych korelacji z pomocniczym aktywem handlowalnym', size = 20)

W porównaniu do opcji call tutaj dla korelacji skrajnie dodatniej niebieska oraz pomarańczowa "rzeczywistość, gdzie aktywo niehandlowalne jest odpowiednio największe i najmniejsze zamieniają się niejako miejscami. Ilość posiadanego aktywa handlowalnego w przypadku symulacji niebieskiej zeruje się z powodu charakteru opcji put, dla symulacji pomarańczowej natomiast wynika to z niepełnego kapitału początkowego, a w konsekwencji obcięcia payoffu zmodyfikowanego identycznie wycenianego aktywa handlowalnego. Dla korelacji skrajnie ujemnej mamy sytuacje dość analogiczną. W obu rozważanych kolumnach zestaw określający rzeczywistość związaną z oscylującą wokół ceny wykonania ceną aktywa niehandlowalnego (i podobnie zachowującym się aktywem handlowalnym) nie zeruje ilości aktywa ze względu na szansę dodatniego payoffu zmodyfikowanego. Korelacja zerowa daje nam wyniki bardzo nieregularne- nie jesteśmy w stanie zabezpieczać opcji na aktywo niehandlowalne niezwiązanym z nim aktywem handlowalnym.

### Analiza wrażliwości zmodyfikowanego payoffu opcji

Kolejne wykresy przedstawiać będą wrażliwość funkcji zmodyfikowanego payoffu dla używanego do zabezpieczania opcji aktywa handlowalnego. Skupimy się na zależności względem rodzaju opcji, ceny wykonania, stosunku zmienności obu aktywów oraz procentu użytej pełnej ceny analogicznej opcji standardowej jako kapitału początkowego. Jeśli dany parametr nie jest w danym przypadku zmienny to przyjmujemy $K = 100$, $\sigma_{NH}=1 \cdot \sigma_{H}=0.3$ oraz $V_0$ w postaci połowy pełnej ceny standardowej opcji. W każdym z rozważanych wykresów przedstawiać będziemy przypadek z korelacją $\rho = +-0.75$.

#### Wrażliwość na cenę wykonania

In [ ]:
rho = 0.75
V0 = BS_call / 2
Ks = [60, 80, 100, 120, 140, 160]
X = pd.DataFrame(np.arange(0,300))
fig, axs = plt.subplots(3, 2, figsize=(10,10), sharey = True)
fig.tight_layout(pad=5.0)
for num, K in tqdm(enumerate(Ks)):
    sims_t = pd.DataFrame(np.ones(300), columns = [0])
    sims_t[1] = np.arange(10, 310) / X0_t
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
    call_nt = Vanilla_on_NonTraded(underlying_nt, K, T, True)
    V0 = Vanilla(underlying_t, K, T, True).get_price(X0_nt) / 2
    call_nt.set_m(V0, X0_t, X0_nt)
    payoff1 = call_nt.payoff_special((X0_t * sims_t), X0_nt)   
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, -rho)
    call_nt = Vanilla_on_NonTraded(underlying_nt, K, T, True)
    call_nt.set_m(V0, X0_t, X0_nt)
    payoff2 = call_nt.payoff_special((X0_t * sims_t), X0_nt)  
    axs[num//2, num%2].plot((X0_t * sims_t).iloc[:,-1], payoff1, label = f'rho = {rho}')
    axs[num//2, num%2].plot((X0_t * sims_t).iloc[:,-1], payoff2, label = f'rho = {-rho}')
    axs[num//2, num%2].legend(loc = 'upper right')
    axs[num//2, num%2].set_title(f'K = {K}')
    axs[num//2, num%2].axvline(K, color = 'black', lw = 4)
    axs[num//2, num%2].set_xlim([10, 310])
    axs[num//2, num%2].set_xlabel('Cena końcowa aktywa handlowalnego')
    axs[num//2, num%2].set_ylabel('Zmodyfikowany payoff')
plt.subplots_adjust(top=0.93)
plt.suptitle(f'Kwantylowy hedging na callu na aktywo niehandlowalne w zależności od ceny wykonania K\nzmodyfikowany payoff', size = 12)

Widzimy, że w przypadku opcji kupna i korelacji dodatniej im większa cena wykonania $K$ tym szerszy przedział niezerowego payoffu. Wynika to z faktu, że ilość symulacji aktywa niehandlowalnego, które osiągną wartości powyżej $K$ w tym przypadku maleje, w konsekwencji możemy tym samym kosztem być zabezpieczeni dla wyższych wartości końcowych. Przypadek z korelacją ujemną jest podobny z dokładnością do odwrócenia payoffu- wydzimu tu jednak że od pewnego momentu nie zabezpieczamy się na coraz mniejsze ceny aktywa handlowego, a jedynie na ten sam przedział, ale z wyższym potencjalnym payoffem. Wynika to z faktu pewnej nieprzeskalowalności nieskończonego potencjalnego wzrostu aktywa niehandlowalnego, a ograniczonej straty instrumentu pomocniczego.

In [ ]:
rho = 0.75
Ks = [60, 80, 100, 120, 140, 160]
X = pd.DataFrame(np.arange(0,300))
fig, axs = plt.subplots(3, 2, figsize=(10,10), sharey = True)
fig.tight_layout(pad=5.0)
for num, K in tqdm(enumerate(Ks)):
    sims_t = pd.DataFrame(np.ones(300), columns = [0])
    sims_t[1] = np.arange(10, 310) / X0_t
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
    V0 = Vanilla(underlying_t, K, T, False).get_price(X0_nt) / 2
    put_nt = Vanilla_on_NonTraded(underlying_nt, K, T, False)
    put_nt.set_m(V0, X0_t, X0_nt)
    payoff1 = put_nt.payoff_special((X0_t * sims_t), X0_nt)   
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, -rho)
    put_nt = Vanilla_on_NonTraded(underlying_nt, K, T, False)
    put_nt.set_m(V0, X0_t, X0_nt)
    payoff2 = put_nt.payoff_special((X0_t * sims_t), X0_nt)  
    axs[num//2, num%2].plot((X0_t * sims_t).iloc[:,-1], payoff1, label = f'rho = {rho}')
    axs[num//2, num%2].plot((X0_t * sims_t).iloc[:,-1], payoff2, label = f'rho = {-rho}')
    axs[num//2, num%2].legend(loc = 'upper right')
    axs[num//2, num%2].set_title(f'K = {K}')
    axs[num//2, num%2].axvline(K, color = 'black', lw = 4)
    axs[num//2, num%2].set_xlim([10, 310])
    axs[num//2, num%2].set_xlabel('Cena końcowa aktywa handlowalnego')
    axs[num//2, num%2].set_ylabel('Zmodyfikowany payoff')
plt.subplots_adjust(top=0.93)
plt.suptitle(f'Kwantylowy hedging na puta na aktywo niehandlowalne w zależności od ceny wykonania K\nzmodyfikowany payoff', size = 12)

Przypadek opcji sprzedaży jest dużo trudniejszy do interpretacji. Zauważalna jest pewna nieidealna symetryczność obu wykresów względem ceny początkowej $X_{0,NH}=X_{0,H}=100$. Skrajnie mała cena wykonania upodabnia wykresy (szczególnie ten z $\rho = 0.75$) do payoffu standardowego opcji put. Dla skrajnie wysokiego $K$ widzimy natomiast, że choć korelacja jest ujemna payoff pomarańczowy znajduje się w całości po lewej stronie ceny wykonania. Wynika to z faktu, że jak już wspomniano ceny obu aktywów są w pewnym stopniu symetryczne do ich początkowej wartości $100$. W konsekwencji wartości aktywa bazowego pomiędzy $60$, a $70$ mogą sugerować wyniki z zakresu $130-140$ instrumentu głównego, a więc takie, gdzie payoff faktyczny byłby istotnie niezerowy i trzeba się na niego przygotować.

#### Wrażliwość na procent ceny analogicznej opcji waniliowej użyty do zabezpieczania

In [ ]:
K = 100
rho = 0.75
alphas = [0.01, 0.1, 0.2, 0.4, 0.6, 0.8, 0.9, 0.99]
X = pd.DataFrame(np.arange(0,300))
fig, axs = plt.subplots(4, 2, figsize=(10,10), sharey = True)
fig.tight_layout(pad=5.0)
for num, alpha in tqdm(enumerate(alphas)):
    sims_t = pd.DataFrame(np.ones(150), columns = [0])
    sims_t[1] = np.arange(50, 200) / X0_t
    V0 = BS_call * alpha
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
    call_nt = Vanilla_on_NonTraded(underlying_nt, K, T, True)
    call_nt.set_m(V0, X0_t, X0_nt)
    payoff1 = call_nt.payoff_special((X0_t * sims_t), X0_nt)   
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, -rho)
    call_nt = Vanilla_on_NonTraded(underlying_nt, K, T, True)
    call_nt.set_m(V0, X0_t, X0_nt)
    payoff2 = call_nt.payoff_special((X0_t * sims_t), X0_nt)  
    axs[num//2, num%2].plot((X0_t * sims_t).iloc[:,-1], payoff1, label = f'rho = {rho}')
    axs[num//2, num%2].plot((X0_t * sims_t).iloc[:,-1], payoff2, label = f'rho = {-rho}')
    axs[num//2, num%2].legend(loc = 'upper right')
    axs[num//2, num%2].set_title(f'alpha = {alpha}')
    axs[num//2, num%2].axvline(K, color = 'black', lw = 4)
    axs[num//2, num%2].set_xlim([50,200])
    axs[num//2, num%2].set_xlabel('Cena końcowa aktywa handlowalnego')
    axs[num//2, num%2].set_ylabel('Zmodyfikowany payoff')
plt.subplots_adjust(top=0.93)
plt.suptitle(f'Kwantylowy hedging na callu(K = {K}) na aktywo niehandlowalne w zależności od procenta użytej ceny zwykłego calla\nzmodyfikowany payoff', size = 12)

W tym wypadku wszystkie wykresy są stosunkowo łatwo interpretowalne- im większy procent ceny opcji standardowej tym na większy przedział końcowej wartości aktywa handlowalnego rzutujący także na przedział aktywa głownego możemy się przygotować. 

In [ ]:
K = 100
rho = 0.75
alphas = [0.01, 0.1, 0.2, 0.4, 0.6, 0.8, 0.9, 0.99]
X = pd.DataFrame(np.arange(0,300))
fig, axs = plt.subplots(4, 2, figsize=(10,10), sharey = True)
fig.tight_layout(pad=5.0)
for num, alpha in tqdm(enumerate(alphas)):
    sims_t = pd.DataFrame(np.ones(150), columns = [0])
    sims_t[1] = np.arange(50, 200) / X0_t
    V0 = BS_put * alpha
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
    put_nt = Vanilla_on_NonTraded(underlying_nt, K, T, False)
    put_nt.set_m(V0, X0_t, X0_nt)
    payoff1 = put_nt.payoff_special((X0_t * sims_t), X0_nt)   
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, -rho)
    put_nt = Vanilla_on_NonTraded(underlying_nt, K, T, False)
    put_nt.set_m(V0, X0_t, X0_nt)
    payoff2 = put_nt.payoff_special((X0_t * sims_t), X0_nt)  
    axs[num//2, num%2].plot((X0_t * sims_t).iloc[:,-1], payoff1, label = f'rho = {rho}')
    axs[num//2, num%2].plot((X0_t * sims_t).iloc[:,-1], payoff2, label = f'rho = {-rho}')
    axs[num//2, num%2].legend(loc = 'upper right')
    axs[num//2, num%2].set_title(f'alpha = {alpha}')
    axs[num//2, num%2].axvline(K, color = 'black', lw = 4)
    axs[num//2, num%2].set_xlim([50,200])
    axs[num//2, num%2].set_xlabel('Cena końcowa aktywa handlowalnego')
    axs[num//2, num%2].set_ylabel('Zmodyfikowany payoff')
plt.subplots_adjust(top=0.93)
plt.suptitle(f'Kwantylowy hedging na puta(K = {K}) na aktywo niehandlowalne w zależności od procenta użytej ceny zwykłego calla\nzmodyfikowany payoff', size = 12)

Opcja put niesie ze sobą wnioski podobne- wraz z rosnącym kapitałem początkowym niezerowy przedział wartości aktywa bazowego, na które mamy się przygotować rośnie. Więcej zabezpieczonych symulacji tego instrumentu oznacza przy tej dość mocnej korelacji również więcej zabezpieczonych symulacji faktycznego payoffu na aktywie niehandlowalnym.

#### Wrażliwość na zmienności obu aktywów

In [ ]:
K = 100
rho = 0.75
V0 = BS_call / 2
sigmas_prop = [0.125, 0.25, 0.5, 1, 2, 4]
X = pd.DataFrame(np.arange(0,300))
fig, axs = plt.subplots(3, 2, figsize=(10,10), sharey = True)
fig.tight_layout(pad=5.0)
for num, prop in tqdm(enumerate(sigmas_prop)):
    sims_t = pd.DataFrame(np.ones(150), columns = [0])
    sims_t[1] = np.arange(50, 200) / X0_t
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_t * prop, underlying_t, rho)
    call_nt = Vanilla_on_NonTraded(underlying_nt, K, T, True)
    call_nt.set_m(V0, X0_t, X0_nt)
    payoff1 = call_nt.payoff_special((X0_t * sims_t), X0_nt)   
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_t * prop, underlying_t, -rho)
    call_nt = Vanilla_on_NonTraded(underlying_nt, K, T, True)
    call_nt.set_m(V0, X0_t, X0_nt)
    payoff2 = call_nt.payoff_special((X0_t * sims_t), X0_nt)  
    axs[num//2, num%2].plot((X0_t * sims_t).iloc[:,-1], payoff1, label = f'rho = {rho}')
    axs[num//2, num%2].plot((X0_t * sims_t).iloc[:,-1], payoff2, label = f'rho = {-rho}')
    axs[num//2, num%2].legend(loc = 'upper right')
    axs[num//2, num%2].set_title(f'sigma_NH = {prop} * sigma_H')
    axs[num//2, num%2].axvline(K, color = 'black', lw = 4)
    axs[num//2, num%2].set_xlim([50,200])
    axs[num//2, num%2].set_xlabel('Cena końcowa aktywa handlowalnego')
    axs[num//2, num%2].set_ylabel('Zmodyfikowany payoff')
plt.subplots_adjust(top=0.93)
plt.suptitle(f'Kwantylowy hedging na callu(K = {K}) na aktywo niehandlowalne w zależności od proporcji zmienności obu aktywów\nzmodyfikowany payoff', size = 12)

Można zauważyć, że im bardziej zmienne jest aktywo niehandlowalne w porównaniu do zabezpieczającego je aktywa handlowalnego tym bardziej "konserwatywny" jest payoff zmodyfikowany. Mówiąc to mamy na myśli fakt, że rośnie on na wartości na niezerowym przedziale oraz staje się coraz bardziej wklęsły. Intuicyjnie owe "wybrzuszenie" stanowi niejako dodatkowy narzut bezpieczeństwa na faktyczny payoff bardziej zmiennego aktywa bazowego. Ze względu na ograniczony kapitał przedziały te wraz z chęcią bycia bardziej konserwatywnym co to samych payoffów węższe.

In [ ]:
K = 100
rho = 0.75
V0 = BS_put / 2
sigmas_prop = [0.125, 0.25, 0.5, 1, 2, 4]
X = pd.DataFrame(np.arange(0,300))
fig, axs = plt.subplots(3, 2, figsize=(10,10), sharey = True)
fig.tight_layout(pad=5.0)
for num, prop in tqdm(enumerate(sigmas_prop)):
    sims_t = pd.DataFrame(np.ones(150), columns = [0])
    sims_t[1] = np.arange(50, 200) / X0_t
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_t * prop, underlying_t, rho)
    put_nt = Vanilla_on_NonTraded(underlying_nt, K, T, False)
    put_nt.set_m(V0, X0_t, X0_nt)
    payoff1 = put_nt.payoff_special((X0_t * sims_t), X0_nt)   
    underlying_nt = NonTradedUnderlying(mu_nt, sigma_t * prop, underlying_t, -rho)
    put_nt = Vanilla_on_NonTraded(underlying_nt, K, T, False)
    put_nt.set_m(V0, X0_t, X0_nt)
    payoff2 = put_nt.payoff_special((X0_t * sims_t), X0_nt)  
    axs[num//2, num%2].plot((X0_t * sims_t).iloc[:,-1], payoff1, label = f'rho = {rho}')
    axs[num//2, num%2].plot((X0_t * sims_t).iloc[:,-1], payoff2, label = f'rho = {-rho}')
    axs[num//2, num%2].legend(loc = 'upper right')
    axs[num//2, num%2].set_title(f'sigma_NH = {prop} * sigma_H')
    axs[num//2, num%2].axvline(K, color = 'black', lw = 4)
    axs[num//2, num%2].set_xlim([50,200])
    axs[num//2, num%2].set_xlabel('Cena końcowa aktywa handlowalnego')
    axs[num//2, num%2].set_ylabel('Zmodyfikowany payoff')
plt.subplots_adjust(top=0.93)
plt.suptitle(f'Kwantylowy hedging na puta(K = {K}) na aktywo niehandlowalne w zależności od proporcji zmienności obu aktywów\nzmodyfikowany payoff', size = 12)

Opcja pull niesie ze sobą w pewnym stopniu podobną charakterystykę- payoffy rosną co do wartości przez dodatkowy narzut bezpieczeństwa na bardziej zmienne niehandlowalne aktywo.

## II. Optymalizacja drugiej funkcji celu

W tej sekcji skupimy się głównie na zmodyfikowanym payoffie, nie zaprezentujemy niestety wyników samego tradingu na przykładzie symulacji. Wynika to z ciężaru obliczeniowego metody szukania stycznej do funkcji $G$, który nakłada się wielokrotnie w przypadku trade'owania, gdzie cana opcji musi zostać wyliczona każdego dnia dwukrotnie w celu obliczenia $\Delta$. Możliwe, choć nadal stosunkowo czasochłonne jest skonstruowanie jednego pełnego tradingu- nie niesie on jednak informacji, które moglibyśmy przenieść na większą liczbę przypadków. Możliwa jest modyfikacja metody na bardziej optymalną lub postawienie na większą liberalność (choć już plasującą się na dość wysokim poziomie) przyjętych parametrów dokładności obliczeń. W poniższym raporcie na taki krok się jednak nie zdecydowaliśmy.

### Parametry wyjściowe

In [ ]:
mu_nt = 0.06
mu_t = 0.06
sigma_nt =  0.3
sigma_t =  0.3
r = 0.05
X0_nt = 100
X0_t = 100
T = 1
K = 100
repeat = 10000
values_per_year = 250

In [ ]:
underlying_t = Underlying(mu_t, sigma_t, r, values_per_year)

### Proste *sanity checki* wyznaczanego przez przypadek skrajny payoffu

In [ ]:
rho = 0.999999
underlying_nt = NonTradedUnderlying(mu_t, sigma_t, underlying_t, rho)
[_, reality_t], [_, reality_nt] = underlying_nt.simulate_together_P(repeat, T)

In [ ]:
call_nt = Vanilla_on_NonTraded(underlying_nt, K, T, True, 'success_ratio')
put_nt = Vanilla_on_NonTraded(underlying_nt, K, T, False, 'success_ratio')

In [ ]:
call_vanilla = Vanilla(underlying_t, K, T, True)
put_vanilla = Vanilla(underlying_t, K, T, False)

#### Cena opcji w świecie Blacka Scholesa- porównanie

In [ ]:
price_MC_nontradable = call_nt.get_MC_price(X0_t, X0_nt)
BS_call = call_vanilla.get_price(X0_t)
print(f'Analitycna cena opcji call:{BS_call:.3f}')
print(f'MC cena opcji na niehandlowalne imitującej zwykłego calla:{price_MC_nontradable:.3f}')
assert abs(price_MC_nontradable - BS_call)/BS_call < 0.1

In [ ]:
price_MC_nontradable = put_nt.get_MC_price(X0_t, X0_nt)
BS_put = put_vanilla.get_price(X0_t)
print(f'Analitycna cena opcji put:{BS_put:.3f}')
print(f'MC cena opcji na niehandlowalne imitującej zwykłego puta:{price_MC_nontradable:.3f}')
assert abs(price_MC_nontradable - BS_put)/BS_put < 0.1

Widzimy, że estymacje ceny opcji na aktywo niehandlowalne w przypadku imitującym opcję waniliową są stosunkowo zbliżone. Potencjalny, mniejszy niż 10% ceny właściwej błąd wynikać może z metody symulacji Monte Carlo. Założony błąd jest tym razem większy ze względu na większą niedokładność obliczeniową definiowania zmodyfikowanego payoffu.

### Analiza różnic pomiędzy zmodyfikowanym payoffem w zależności od tego, która funkcja celu ma być optymalizowana

Poniżej zaprezentujemy porównania zmodyfikowanego payoffu opcji na aktywo niehandlowalne w zależności, czy zależy nam na optymalizacji pierwszej funkcji celu tj. prawdopodobieństwa sukcesu, czy też drugiej (tzw. *success ratio*). Analizy te będą zależne od przyjętej korelacji pomiędzy aktywami, procentu przekazanej do zabezpieczania ceny standardowego odpowiednika opcji, proporcji pomiędzy zmiennością aktywów oraz ceny wykonania.

#### Wrażliwość na cenę wykonania

In [ ]:
fig, axs = plt.subplots(3,5, figsize = (25, 15), sharey = 'row')
fig.tight_layout(pad=5.5)
plt.suptitle(f'Różnice między zmodyfikowanymi payoffami kwantylowego hedgingu calla\nw zależności od ceny wykonania K oraz korelacji z aktywem handlowalnym', fontsize= 18)
plt.subplots_adjust(top=0.9)
for idy, rho in tqdm(enumerate([0.75, 0.5, 0, -0.5, -0.75])):
    for idx, K in enumerate([60, 100, 140]):
        underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
        sims_t = pd.DataFrame(np.ones(152), columns = [0])
        sims_t[1] = np.arange(49, 201) / X0_t
        call_nt_sp = Vanilla_on_NonTraded(underlying_nt, K, T, True)
        V0 = Vanilla(underlying_t, K, T, True).get_price(X0_nt) / 2
        call_nt_sp.set_m(V0, X0_t, X0_nt)
        payoff_sp = call_nt_sp.payoff_special((X0_t * sims_t), X0_nt)

        call_nt_sr = Vanilla_on_NonTraded(underlying_nt, K, T, True, 'success_ratio')
        call_nt_sr.set_m(V0, X0_t, X0_nt)
        payoff_sr = call_nt_sr.payoff_special((X0_t * sims_t), X0_nt)

        axs[idx, idy].plot((X0_t * sims_t).iloc[:,-1], payoff_sp, lw = 3, alpha = 0.5, color = 'red', label = 'max pstwo sukcesu')
        axs[idx, idy].plot((X0_t * sims_t).iloc[:,-1], payoff_sr, lw = 3, alpha = 0.5, color = 'green', label = 'max success ratio')
        axs[idx, idy].legend(loc = 'upper right')
        axs[idx, idy].set_title(f'rho = {rho}, K = {K}', fontsize = 13)
        axs[idx, idy].set_xlim([50, 200])    
        axs[idx, idy].axvline(K, color = 'black', lw = 4)
        axs[idx, idy].set_xlabel('Końcowa cena aktywa handlowalnego')
        axs[idx, idy].set_ylabel('Payoff zmodyfikowany', fontsize = 12)    
        axs[idx, idy].set_ylabel('Cena końcowa aktywa handlowalnego', fontsize = 12)    

W przypadku opcji call pierwszą narzucającą się obserwacją jest to, że zmodyfikowany payoff w przypadku optymalizacji *success ratio* jest funkcją zdecydowanie bardziej "gładszą" i w konsekwencji obejmującą większy zakres symulacji. Jednocześnie- ze względu na ograniczony kapitał początkowy payoff ten sięga wartości niższych. Możemy to interpretować jako próbę conajmniej częściowego zabezpieczenia większego zakresu symulacji. Wynika to z formy drugiej funkcji celu, która maksymalizuje także stosunek końcowego stanu portfela przed wypłaceniem payoffu faktycznego do tego właśnie payoffu w przypadku scenariuszy, na które nie byliśmy w pełni przygotowani. Wraz ze wzrostem ceny wykonania oba przypadki payoffów dla aktywa handlowalnego przesuwają swoją masę w prawo kiedy korelacja jest dodatnia oraz w lewo w przypadku przeciwnym.

In [ ]:
fig, axs = plt.subplots(3,5, figsize = (25, 15), sharey = 'row')
fig.tight_layout(pad=5.5)
plt.suptitle(f'Różnice między zmodyfikowanymi payoffami kwantylowego hedgingu puta\nw zależności od ceny wykonania K oraz korelacji z aktywem handlowalnym', fontsize= 18)
plt.subplots_adjust(top=0.9)
V0 = BS_put / 2
for idy, rho in tqdm(enumerate([0.75, 0.5, 0, -0.5, -0.75])):
    for idx, K in enumerate([60, 100, 140]):
        underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
        sims_t = pd.DataFrame(np.ones(152), columns = [0])
        sims_t[1] = np.arange(49, 201) / X0_t
        put_nt_sp = Vanilla_on_NonTraded(underlying_nt, K, T, False)
        V0 = Vanilla(underlying_t, K, T, False).get_price(X0_nt) / 2
        put_nt_sp.set_m(V0, X0_t, X0_nt)
        put_sp = put_nt_sp.payoff_special((X0_t * sims_t), X0_nt)

        put_nt_sr = Vanilla_on_NonTraded(underlying_nt, K, T, False, 'success_ratio')
        put_nt_sr.set_m(V0, X0_t, X0_nt)
        payoff_sr = put_nt_sr.payoff_special((X0_t * sims_t), X0_nt)

        axs[idx, idy].plot((X0_t * sims_t).iloc[:,-1], payoff_sp, lw = 3, alpha = 0.5, color = 'red', label = 'max pstwo sukcesu')
        axs[idx, idy].plot((X0_t * sims_t).iloc[:,-1], payoff_sr, lw = 3, alpha = 0.5, color = 'green', label = 'max success ratio')
        axs[idx, idy].legend(loc = 'upper right')
        axs[idx, idy].set_title(f'rho = {rho}, K = {K}', fontsize = 13)
        axs[idx, idy].set_xlim([50, 200])    
        axs[idx, idy].axvline(K, color = 'black', lw = 4)
        axs[idx, idy].set_xlabel('Końcowa cena aktywa handlowalnego')
        axs[idx, idy].set_ylabel('Payoff zmodyfikowany', fontsize = 12)    
        axs[idx, idy].set_ylabel('Cena końcowa aktywa handlowalnego', fontsize = 12)    

Zmodyfikowane payoffy dla opcji put są analogicznie od siebie zależne, co dla opcji call. Tym razem wraz ze wzrostem ceny wykonania payoffy te przesuwają masę ku wartościom mniejszym kiedy korelacja jest dodatnia lub większym w przypadku $\rho <0$. Kiedy aktywa są nieskorelowane payoffy "próbują" zabezpieczyć się na losowość aktywa niehandlowalnego- nie ma tu tak dużej zależności od ceny wykonania $K$.

#### Wrażliwość na procent ceny analogicznej opcji waniliowej użyty do zabezpieczania

In [ ]:
fig, axs = plt.subplots(3,5, figsize = (25, 15), sharey = 'row')
fig.tight_layout(pad=5.5)
plt.suptitle(f'Różnice między zmodyfikowanymi payoffami kwantylowego hedgingu calla(K = {K})\nw zależności od procentu alpha użytej ceny oraz korelacji z aktywem handlowalnym', fontsize= 18)
plt.subplots_adjust(top=0.9)
V0 = BS_call
for idy, rho in tqdm(enumerate([0.75, 0.5, 0, -0.5, -0.75])):
    for idx, alpha in enumerate([1, 0.5, 0.1]):
        underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
        sims_t = pd.DataFrame(np.ones(152), columns = [0])
        sims_t[1] = np.arange(49, 201) / X0_t
        call_nt_sp = Vanilla_on_NonTraded(underlying_nt, K, T, True)
        call_nt_sp.set_m(alpha * V0, X0_t, X0_nt)
        payoff_sp = call_nt_sp.payoff_special((X0_t * sims_t), X0_nt)

        call_nt_sr = Vanilla_on_NonTraded(underlying_nt, K, T, True, 'success_ratio')
        call_nt_sr.set_m(alpha * V0, X0_t, X0_nt)
        payoff_sr = call_nt_sr.payoff_special((X0_t * sims_t), X0_nt)

        axs[idx, idy].plot((X0_t * sims_t).iloc[:,-1], payoff_sp, lw = 3, alpha = 0.5, color = 'red', label = 'max pstwo sukcesu')
        axs[idx, idy].plot((X0_t * sims_t).iloc[:,-1], payoff_sr, lw = 3, alpha = 0.5, color = 'green', label = 'max success ratio')
        axs[idx, idy].legend(loc = 'upper right')
        axs[idx, idy].set_title(f'rho = {rho}, alpha = {alpha}', fontsize = 13)
        axs[idx, idy].set_xlim([50, 200])    
        axs[idx, idy].axvline(K, color = 'black', lw = 4)
        axs[idx, idy].set_xlabel('Końcowa cena aktywa handlowalnego')
        axs[idx, idy].set_ylabel('Payoff zmodyfikowany', fontsize = 12)    
        axs[idx, idy].set_ylabel('Cena końcowa aktywa handlowalnego', fontsize = 12)    

Zauważmy, że w przypadku gdy kapitał początkowy jest większy wykresy zmodyfikowanego payoffu dla obu funkcji celu są stosunkowo podobne. P-stwo sukcesu stanowiące jeden składnik *success ratio* dominuje tutaj i drugi składnik związany ze scenariuszami niezabezpieczonymi nie modyfikuje znacznie payoffu. Zmienia się to wraz ze spadkiem procentu użytej do hedgingu ceny standardowej wersji rozważanej opcji. Podczas gdy payoff zmodyfikowany dla prawdopodobieństwa sukcesu w przypadkach niezerowej korelacji po prostu "zwęża się", na coraz mniejszy przedział- payoff dla *success ratio* rozszerza się na coraz więcej przypadków scenariuszy, których pierwszy nie był w stanie zabezpieczyć. 

In [ ]:
fig, axs = plt.subplots(3,5, figsize = (25, 15), sharey = 'row')
fig.tight_layout(pad=5.5)
plt.suptitle(f'Różnice między zmodyfikowanymi payoffami kwantylowego hedgingu puta(K = {K})\nw zależności od procentu alpha użytej ceny oraz korelacji z aktywem handlowalnym', fontsize= 18)
plt.subplots_adjust(top=0.9)
V0 = BS_put
for idy, rho in tqdm(enumerate([0.75, 0.5, 0, -0.5, -0.75])):
    for idx, alpha in enumerate([1, 0.5, 0.1]):
        underlying_nt = NonTradedUnderlying(mu_nt, sigma_nt, underlying_t, rho)
        sims_t = pd.DataFrame(np.ones(152), columns = [0])
        sims_t[1] = np.arange(49, 201) / X0_t
        put_nt_sp = Vanilla_on_NonTraded(underlying_nt, K, T, False)
        put_nt_sp.set_m(alpha * V0, X0_t, X0_nt)
        payoff_sp = put_nt_sp.payoff_special((X0_t * sims_t), X0_nt)

        put_nt_sr = Vanilla_on_NonTraded(underlying_nt, K, T, False, 'success_ratio')
        put_nt_sr.set_m(alpha * V0, X0_t, X0_nt)
        payoff_sr = put_nt_sr.payoff_special((X0_t * sims_t), X0_nt)

        axs[idx, idy].plot((X0_t * sims_t).iloc[:,-1], payoff_sp, lw = 3, alpha = 0.5, color = 'red', label = 'max pstwo sukcesu')
        axs[idx, idy].plot((X0_t * sims_t).iloc[:,-1], payoff_sr, lw = 3, alpha = 0.5, color = 'green', label = 'max success ratio')
        axs[idx, idy].legend(loc = 'upper right')
        axs[idx, idy].set_title(f'rho = {rho}, alpha = {alpha}', fontsize = 13)
        axs[idx, idy].set_xlim([50, 200])    
        axs[idx, idy].axvline(K, color = 'black', lw = 4)
        axs[idx, idy].set_xlabel('Końcowa cena aktywa handlowalnego')
        axs[idx, idy].set_ylabel('Payoff zmodyfikowany', fontsize = 12)    
        axs[idx, idy].set_ylabel('Cena końcowa aktywa handlowalnego', fontsize = 12)    

Wnioski w przypadku opcji put są zbliżone- payoff zaznaczony tu kolorem zielony próbuje sięgnąć dużo większej liczby scenariuszy zmniejszając jednocześnie oczekiwany od nich payoff.

#### Wrażliwość na zmienności obu aktywów

In [ ]:
fig, axs = plt.subplots(3,5, figsize = (25, 15), sharey = 'row')
fig.tight_layout(pad=5.5)
K = 100
plt.suptitle(f'Różnice między zmodyfikowanymi payoffami kwantylowego hedgingu calla(K = {K})\nw zależności od proporcji zmienności aktywów oraz korelacji z aktywem handlowalnym', fontsize= 18)
plt.subplots_adjust(top=0.9)
V0 = BS_call / 2
for idy, rho in tqdm(enumerate([0.75, 0.5, 0, -0.5, -0.75])):
    for idx, prop in enumerate([1/2, 1, 2]):
        underlying_nt = NonTradedUnderlying(mu_nt, prop * sigma_t, underlying_t, rho)
        sims_t = pd.DataFrame(np.ones(152), columns = [0])
        sims_t[1] = np.arange(49, 201) / X0_t
        call_nt_sp = Vanilla_on_NonTraded(underlying_nt, K, T, True)
        call_nt_sp.set_m(V0, X0_t, X0_nt)
        payoff_sp = call_nt_sp.payoff_special((X0_t * sims_t), X0_nt)

        call_nt_sr = Vanilla_on_NonTraded(underlying_nt, K, T, True, 'success_ratio')
        call_nt_sr.set_m(V0, X0_t, X0_nt)
        payoff_sr = call_nt_sr.payoff_special((X0_t * sims_t), X0_nt)

        axs[idx, idy].plot((X0_t * sims_t).iloc[:,-1], payoff_sp, lw = 3, alpha = 0.5, color = 'red', label = 'max pstwo sukcesu')
        axs[idx, idy].plot((X0_t * sims_t).iloc[:,-1], payoff_sr, lw = 3, alpha = 0.5, color = 'green', label = 'max success ratio')
        axs[idx, idy].legend(loc = 'upper right')
        axs[idx, idy].set_title(f'rho = {rho}, sigma_NH = {prop}*sigma_H', fontsize = 13)
        axs[idx, idy].set_xlim([50, 200])   
        axs[idx, idy].axvline(K, color = 'black', lw = 4)
        axs[idx, idy].set_xlabel('Końcowa cena aktywa handlowalnego')
        axs[idx, idy].set_ylabel('Payoff zmodyfikowany', fontsize = 12)    
        axs[idx, idy].set_ylabel('Cena końcowa aktywa handlowalnego', fontsize = 12)    

W przypadku optymalizacji obu funkcji celu wzrost w zmienności aktywa niehandlowalnego skutkuje w większej wklęsłości zmodyfikowanych payoffów. Jest to widoczne bardziej w przypadku payoffów, dla maksymalizacji prawdopodobieństwa sukcesu.

In [ ]:
fig, axs = plt.subplots(3,5, figsize = (25, 15), sharey = 'row')
fig.tight_layout(pad=5.5)
K = 100
plt.suptitle(f'Różnice między zmodyfikowanymi payoffami kwantylowego hedgingu puta(K = {K})\nw zależności od proporcji zmienności aktywów oraz korelacji z aktywem handlowalnym', fontsize= 18)
plt.subplots_adjust(top=0.9)
V0 = BS_put / 2
for idy, rho in tqdm(enumerate([0.75, 0.5, 0, -0.5, -0.75])):
    for idx, prop in enumerate([1/2, 1, 2]):
        underlying_nt = NonTradedUnderlying(mu_nt, prop * sigma_t, underlying_t, rho)
        sims_t = pd.DataFrame(np.ones(152), columns = [0])
        sims_t[1] = np.arange(49, 201) / X0_t
        put_nt_sp = Vanilla_on_NonTraded(underlying_nt, K, T, False)
        put_nt_sp.set_m(V0, X0_t, X0_nt)
        payoff_sp = put_nt_sp.payoff_special((X0_t * sims_t), X0_nt)

        put_nt_sr = Vanilla_on_NonTraded(underlying_nt, K, T, False, 'success_ratio')
        put_nt_sr.set_m(V0, X0_t, X0_nt)
        payoff_sr = put_nt_sr.payoff_special((X0_t * sims_t), X0_nt)

        axs[idx, idy].plot((X0_t * sims_t).iloc[:,-1], payoff_sp, lw = 3, alpha = 0.5, color = 'red', label = 'max pstwo sukcesu')
        axs[idx, idy].plot((X0_t * sims_t).iloc[:,-1], payoff_sr, lw = 3, alpha = 0.5, color = 'green', label = 'max success ratio')
        axs[idx, idy].legend(loc = 'upper right')
        axs[idx, idy].set_title(f'rho = {rho}, sigma_NH = {prop}*sigma_H', fontsize = 13)
        axs[idx, idy].set_xlim([50, 200]) 
        axs[idx, idy].axvline(K, color = 'black', lw = 4)
        axs[idx, idy].set_xlabel('Końcowa cena aktywa handlowalnego')
        axs[idx, idy].set_ylabel('Payoff zmodyfikowany', fontsize = 12)    
        axs[idx, idy].set_ylabel('Cena końcowa aktywa handlowalnego', fontsize = 12)    

Wnioski dla opcji sprzedaży są zbliżone.